## Identifying and resolving conflicts in a batch of VI files

ELGs - part 4 is sets 16 to 20

There are 798 visual inspections of a total of 250 unique objects

65 conflicts

In [2]:
save_filename='truth_table_ELGs_sets16_20.txt'

In [3]:
import os, sys, glob
import numpy as np

from astropy.io import fits
from astropy.table import Table, join, vstack
from astropy.io import fits

import pandas as pd
import fnmatch

import desispec.io

#import desispec
# library location will change ..
sys.path.append("/global/homes/r/rtojeiro/prospect/prospect/py")
from prospect import utils_specviewer,plotframes
import matplotlib.pyplot as plt 

In [4]:
def find_conflicts(vi_gp):
    vi_conflict = vi_gp.filter(lambda x: ( ( (x['VI class'].max()-x['VI class'].min()) >= 2) 
                       | ( (x['best redshift'].max() - x['best redshift'].min()) / (1+x['best redshift'].min()) > 0.0033 ) 
                       | (not all(i == x['best spectype'].iloc[0] for i in x['best spectype'])) )
                       & (x['VI class'].max() > 2)
                       & (len(x) >= 2)) #x is a group by TargetID)
    return vi_conflict

In [5]:
tiledir   = '/global/cfs/cdirs/desi/spectro/redux/daily/tiles/'
tiles = ['67230']
nights = ['20200315']
petals = ['0','1', '2', '3', '4', '5', '6' ,'7', '8', '9']
obs_db = utils_specviewer.make_targetdict(tiledir, petals=petals, tiles=tiles) # tiles = optional 

In [6]:
pd.set_option('display.max_rows', 20)

In [7]:
#set to directory with all the VI files to merge
VI_dir = os.environ['HOME']+'/SV/VI_files/SV0/ELG/'

In [8]:
#read a set of files 
all_files = os.listdir(VI_dir)
vi_files=[]

for i in range(16,21):
    pattern = "desi*_"+str(i)+"_*.csv"
    print(pattern)
    for entry in all_files:
        if fnmatch.fnmatch(entry, pattern):
            vi_files.append(entry)

vi_files

desi*_16_*.csv
desi*_17_*.csv
desi*_18_*.csv
desi*_19_*.csv
desi*_20_*.csv


['desi-vi_SV0_ELG_tile67230_night20200315_16_DRP.csv',
 'desi-vi_SV0_ELG_tile67230_night20200315_16_VRK.csv',
 'desi-vi_SV0_ELG_tile67230_night20200315_16_TWL.csv',
 'desi-vi_SV0_ELG_tile67230_night20200315_17_TWL.csv',
 'desi-vi_SV0_ELG_tile67230_night20200315_17_SEM.csv',
 'desi-vi_SV0_ELG_tile67230_night20200315_17_VRK.csv',
 'desi-vi_SV0_ELG_tile67230_night20200315_18_TWL.csv',
 'desi-vi_SV0_ELG_tile67230_night20200315_18_SEM.csv',
 'desi-vi_SV0_ELG_tile67230_night20200315_18_PNT.csv',
 'desi-vi_SV0_ELG_tile67230_night20200315_19_PNT.csv',
 'desi-vi_SV0_ELG_tile67230_night20200315_19_VRK.csv',
 'desi-vi_SV0_ELG_tile67230_night20200315_19_ACE.csv',
 'desi-vi_SV0_ELG_tile67230_night20200315_19_TWL.csv',
 'desi-vi_SV0_ELG_tile67230_night20200315_20_TWL.csv',
 'desi-vi_SV0_ELG_tile67230_night20200315_20_ACE.csv',
 'desi-vi_SV0_ELG_tile67230_night20200315_20_PNT.csv']

In [9]:
vi = pd.read_csv(VI_dir + vi_files[0], delimiter = " , ", engine='python')

for i in range(1,len(vi_files)):
    print(vi_files[i])
    vi2 = pd.read_csv(VI_dir + vi_files[i], delimiter = " , ", engine='python')
    vi = vi.append(vi2, ignore_index=True)
    
#make groups of visual inspections, grouped by unique objects, and state number of single and multiple VIs
vi_gp = vi.groupby(['TargetID'])
print('There are ' + str(len(vi)) + ' visual inspections of a total of ' + str(len(vi_gp)) + ' unique objects')
print('That gives ' + str(len(vi)/len(vi_gp)) + ' inspections per object.')

desi-vi_SV0_ELG_tile67230_night20200315_16_VRK.csv
desi-vi_SV0_ELG_tile67230_night20200315_16_TWL.csv
desi-vi_SV0_ELG_tile67230_night20200315_17_TWL.csv
desi-vi_SV0_ELG_tile67230_night20200315_17_SEM.csv
desi-vi_SV0_ELG_tile67230_night20200315_17_VRK.csv
desi-vi_SV0_ELG_tile67230_night20200315_18_TWL.csv
desi-vi_SV0_ELG_tile67230_night20200315_18_SEM.csv
desi-vi_SV0_ELG_tile67230_night20200315_18_PNT.csv
desi-vi_SV0_ELG_tile67230_night20200315_19_PNT.csv
desi-vi_SV0_ELG_tile67230_night20200315_19_VRK.csv
desi-vi_SV0_ELG_tile67230_night20200315_19_ACE.csv
desi-vi_SV0_ELG_tile67230_night20200315_19_TWL.csv
desi-vi_SV0_ELG_tile67230_night20200315_20_TWL.csv
desi-vi_SV0_ELG_tile67230_night20200315_20_ACE.csv
desi-vi_SV0_ELG_tile67230_night20200315_20_PNT.csv
There are 798 visual inspections of a total of 250 unique objects
That gives 3.192 inspections per object.


In [10]:
vi.keys()

Index(['TargetID', 'ExpID', 'Spec version', 'Redrock version',
       'Redrock spectype', 'Redrock z', 'VI scanner', 'VI class', 'VI issue',
       'VI z', 'VI spectype', 'VI comment'],
      dtype='object')

### Merge with zbest files
Add: fiberID, delta_chi2, flux information,.. anything else?


In [11]:

#read in fibermap info, loop over the files for all the petals
tf = Table.read(tiledir+'/'+tiles[0] + '/'+nights[0]+'/zbest-'+str(petals[0])+'-'+str(tiles[0])+'-'+nights[0]+'.fits',hdu='FIBERMAP')
tspec = Table.read(tiledir+'/'+tiles[0] + '/'+nights[0]+'/zbest-'+str(petals[0])+'-'+str(tiles[0])+'-'+nights[0]+'.fits',hdu='ZBEST')
for i in range(1,len(petals)):
    tn = Table.read(tiledir+'/'+tiles[0] + '/'+nights[0]+'/zbest-'+str(petals[i])+'-'+str(tiles[0])+'-'+nights[0]+'.fits',hdu='ZBEST')
    tnf = Table.read(tiledir+'/'+tiles[0] + '/'+nights[0]+'/zbest-'+str(petals[i])+'-'+str(tiles[0])+'-'+nights[0]+'.fits',hdu='FIBERMAP')
    tspec = vstack([tspec,tn])
    tf = vstack([tf,tnf])

tspec_df = tspec['TARGETID','DELTACHI2' ].to_pandas()
tf_df = tf['TARGETID','FIBER','FLUX_G','FLUX_R','FLUX_Z','FIBERFLUX_G','FIBERFLUX_R','FIBERFLUX_Z'].to_pandas()

tf_df = tf_df.rename(columns={"TARGETID": "TargetID"})
tspec_df = tspec_df.rename(columns={"TARGETID": "TargetID"})

vi = vi.merge(tf_df, how='left', on='TargetID')
vi = vi.merge(tspec_df, how='left', on='TargetID')

In [12]:
print(vi.keys())
print(len(vi))

Index(['TargetID', 'ExpID', 'Spec version', 'Redrock version',
       'Redrock spectype', 'Redrock z', 'VI scanner', 'VI class', 'VI issue',
       'VI z', 'VI spectype', 'VI comment', 'FIBER', 'FLUX_G', 'FLUX_R',
       'FLUX_Z', 'FIBERFLUX_G', 'FIBERFLUX_R', 'FIBERFLUX_Z', 'DELTACHI2'],
      dtype='object')
798


### Adding a bunch of useful columns

In [13]:
#make new column with best redshift estimate for each VI - take VI redshift if available, else take Redrock redshift. 
#I am always assuming that the VI redshift, if provided, trumps over the Redrock redshift. 
vi['best redshift'] = vi['VI z']
vi.loc[vi['best redshift']=='--', 'best redshift'] = vi.loc[vi['best redshift']=='--', 'Redrock z']
vi.loc[vi['best redshift']=='Target ', 'best redshift'] = 0 #catch someone who wrote in VI z field
vi['best redshift'] = vi['best redshift'].astype(float)

In [14]:
#make new column with best spectype estimate for each VI - take VI spectype if available, else take Redrock spectype 
#I am always assuming that the VI redshift, if provided, trumps over the Redrock redshift. 
vi['best spectype'] = vi['VI spectype']
vi.loc[vi['best spectype']=='--', 'best spectype'] = vi.loc[vi['best spectype']=='--', 'Redrock spectype']

In [15]:
#add new columns, holding the mean of the flags and the maximum difference in flag classification
vi['vi_combined_flag'] = vi.groupby('TargetID')['VI class'].transform('mean')
vi['vi_diff'] = vi.groupby('TargetID')['VI class'].transform(lambda x: ( x.max()-x.min()) )

In [16]:
#add new column, difference in 'best redshift'
vi['dz'] = vi.groupby('TargetID')['best redshift'].transform(lambda x: ( (x.max() - x.min()) / (1+x.min()) ))
#add new column with the mean of best redshift if they're within 0.0033*(1+z)
vi.loc[vi.dz < 0.0033, 'vi_combined_z'] = vi.loc[vi.dz < 0.0033].groupby('TargetID')['best redshift'].transform('mean')
#keep as redrock z is difference is greater. These will be manually resolved if at least one of the inspectors flags a z as flag >2
vi.loc[vi.dz >= 0.0033, 'vi_combined_z'] = vi.loc[vi.dz >= 0.0033, 'Redrock z']

In [17]:
#add new column, with all comments concatenated
vi['all VI comments'] = vi.groupby('TargetID')['VI comment'].transform(lambda x: '|'.join(x))

In [18]:
#add new column, with the number of VI inspections for each object
vi['N_VI'] = vi.groupby('TargetID')['TargetID'].transform('count')

In [19]:
#add new column to hold comments from merger if needed
vi['merger comment'] = 'none'

In [20]:
#check all the new columns (keys) have been added correctly
display(vi)
print(vi.keys())

,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
0,35185760080494861,-1,0.33.0.dev3890,0,GALAXY,0.8069,DRP,4,--,--,...,911.096817,0.8069,GALAXY,4.000000,0,0.000000,0.806900,--|--|--,3,none
1,35185760080494926,-1,0.33.0.dev3890,0,GALAXY,1.6258,DRP,0,--,--,...,4.442083,1.6258,GALAXY,1.000000,3,0.000533,1.626267,--|doublet with one very strong line at the ed...,3,none
2,35185760080495168,-1,0.33.0.dev3890,0,GALAXY,1.2225,DRP,2,--,--,...,83.465970,1.2225,GALAXY,2.333333,1,0.000000,1.222500,--|--| [OII] on top of sky lines,3,none
3,35185760080495716,-1,0.33.0.dev3890,0,GALAXY,1.5493,DRP,1,--,--,...,62.264520,1.5493,GALAXY,2.333333,2,0.000000,1.549300,--| resolved doublet in noisy region though| [...,3,none
4,35185760080495820,-1,0.33.0.dev3890,0,GALAXY,0.9793,DRP,3,--,--,...,231.904559,0.9793,GALAXY,3.666667,1,0.000000,0.979300,--|--|--,3,none
5,35185760080495972,-1,0.33.0.dev3890,0,GALAXY,1.6282,DRP,0,--,--,...,31.021876,1.6282,GALAXY,1.333333,2,0.000000,1.628200,--| strong doublet but at the edge of the nir ...,3,none
6,35185760072107938,-1,0.33.0.dev3890,0,GALAXY,1.6966,DRP,1,--,--,...,2.103210,1.6966,GALAXY,0.333333,1,0.000000,1.696600,--|continuum only + no signal| Low S/N,3,none
7,35185760072108290,-1,0.33.0.dev3890,0,GALAXY,1.5248,DRP,2,--,--,...,13.945422,1.5248,GALAXY,0.666667,2,0.000000,1.524800,--| dubious fit (very weak doublet in noisy re...,3,none
8,35185760072108293,-1,0.33.0.dev3890,0,GALAXY,1.1186,DRP,4,--,--,...,363.574735,1.1186,GALAXY,4.000000,0,0.000000,1.118600,--|--|--,3,none
9,35185760072108452,-1,0.33.0.dev3890,0,GALAXY,0.7814,DRP,0,S,--,...,28.326496,0.7814,GALAXY,0.000000,0,0.000000,0.781400,--| calibration problem at 6400AA| Calibration...,3,none


Index(['TargetID', 'ExpID', 'Spec version', 'Redrock version',
       'Redrock spectype', 'Redrock z', 'VI scanner', 'VI class', 'VI issue',
       'VI z', 'VI spectype', 'VI comment', 'FIBER', 'FLUX_G', 'FLUX_R',
       'FLUX_Z', 'FIBERFLUX_G', 'FIBERFLUX_R', 'FIBERFLUX_Z', 'DELTACHI2',
       'best redshift', 'best spectype', 'vi_combined_flag', 'vi_diff', 'dz',
       'vi_combined_z', 'all VI comments', 'N_VI', 'merger comment'],
      dtype='object')


Get a table that holds only the objects that have been inspected more than once, and for which the individual VI classifications differ by 2 or more, or delta z / (1 + z) > 0.0033, or there is disagreement in best spectype (these are the conflicts to resolve)

In [21]:
vi_gp = vi.groupby(['TargetID'])
#vi_conflict = vi_gp.filter(lambda x: ( ( (x['VI class'].max()-x['VI class'].min()) >= 2) 
#                       | ( (x['best redshift'].max() - x['best redshift'].min()) / (1+x['best redshift'].min()) > 0.0033 ) 
#                       | (not all(i == x['best spectype'].iloc[0] for i in x['best spectype'])) )
#                       & (len(x) >= 2)) #x is a group by TargetID
vi_conflict = find_conflicts(vi_gp)

Get the target IDs of the problematic objects and display in table form for a quick summary:

In [22]:
unique_targets = np.unique(vi_conflict['TargetID'].tolist())
print('Targets with problematic VI: ', unique_targets)
print('Total number of conflicts to resolve: ', len(unique_targets))

Targets with problematic VI:  [35185760072111453 35185760076300581 35185760076302567 35185760076303437
 35185760076303740 35185760076304803 35185760076305772 35185760080494926
 35185760080495716 35185760084690463 35185760084692144 35185760084692854
 35185760093078527 35185760093081089 35185760093081557 35185760097272055
 35185760097272551 35185760097273421 35185760097274623 35185760097274678
 35185760097275227 35185760097275715 35185760097276144 35185760101466913
 35185760101467485 35185760101467682 35185760101468803 35185760101468965
 35185760101469754 35185760101469821 35185760101469927 35185760101470583
 35185760101471222 35185760101471315 35185760105661565 35185760105662202
 35185760105662698 35185760105663384 35185760105663463 35185760105664072
 35185760105664248 35185760105664964 35185760105665118 35185760105665215
 35185760105665533 35185766095130548 35185766099323497 35185766099324463
 35185766099324852 35185766099324872 35185766099325112 35185766099325192
 35185766099325339 35

In [23]:
unique_targets[1]

35185760076300581

## This is where I resolve things manually - with care!!
### I think it's better to keep it in a notebook, as typos can be backtracked rather than a single manual edit of a text file

We edit either 'VI class', or 'best redshift' to resolve conflict. At the end, we look for conflicts again and we should find none.


In [24]:
#function to display the conflict in table format and open a prospect window
def display_conflict(conflict_id, show_spectra=None):
    
    #first, remind myself of the problem:
    display(vi[vi.TargetID==unique_targets[conflict_id]][['TargetID', 
       'Redrock spectype', 'Redrock z', 'VI scanner', 'VI class', 'VI issue',
       'VI z', 'VI spectype', 'VI comment',
       'best redshift', 'best spectype', 'vi_combined_flag', 'vi_diff',
       'vi_combined_z', 'dz', 'all VI comments', 'N_VI', 'merger comment']])

    # VI interface in notebook
    if show_spectra:
        spectra, zcat= utils_specviewer.load_spectra_zcat_from_targets([unique_targets[conflict_id]], tiledir, obs_db)
        plotframes.plotspectra(spectra, zcatalog=zcat, title='Target_select', notebook=True, mask_type='CMX_TARGET',with_vi_widgets=False)

In [25]:
#first, keep a safe copy of the original dataframe
vi_safe = vi.copy()

We will inspect each conflict on a prospect window, and resolve each conflict in turn

In [26]:
#Keep track of the conflicts by conflict_id

#copy this text to a new cell to display the conflict
conflict_id=0
display_conflict(conflict_id, show_spectra=False)

,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
19,35185760072111453,GALAXY,1.3034,DRP,2,--,--,--,--,1.3034,GALAXY,3.0,2,1.3034,0.0,--|--| Triple peak [OII],3,none
69,35185760072111453,GALAXY,1.3034,vrk,3,--,--,--,--,1.3034,GALAXY,3.0,2,1.3034,0.0,--|--| Triple peak [OII],3,none
119,35185760072111453,GALAXY,1.3034,TWL,4,--,--,--,Triple peak [OII],1.3034,GALAXY,3.0,2,1.3034,0.0,--|--| Triple peak [OII],3,none


In [27]:
#copy this text to a new cell to resolve the conflict
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 3
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'flag disagreement. strong OII broadened but not quite resolved. continuum break. z secure.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
19,35185760072111453,-1,0.33.0.dev3890,0,GALAXY,1.3034,DRP,3,--,--,...,101.047926,1.3034,GALAXY,3.0,2,0.0,1.3034,--|--| Triple peak [OII],3,flag disagreement. strong OII broadened but no...
69,35185760072111453,-1,0.33.0.dev3890,0,GALAXY,1.3034,vrk,3,--,--,...,101.047926,1.3034,GALAXY,3.0,2,0.0,1.3034,--|--| Triple peak [OII],3,flag disagreement. strong OII broadened but no...
119,35185760072111453,-1,0.33.0.dev3890,0,GALAXY,1.3034,TWL,3,--,--,...,101.047926,1.3034,GALAXY,3.0,2,0.0,1.3034,--|--| Triple peak [OII],3,flag disagreement. strong OII broadened but no...


conflict ID =  0


In [28]:
#copy this text to a new cell to display the conflict
conflict_id=1
display_conflict(conflict_id, show_spectra=False)


,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
20,35185760076300581,GALAXY,1.5667,DRP,1,--,--,--,--,1.5667,GALAXY,2.0,2,1.5667,0.0,--|unsecure doublet in noisy region + no signa...,3,none
70,35185760076300581,GALAXY,1.5667,vrk,2,--,--,--,unsecure doublet in noisy region + no signal e...,1.5667,GALAXY,2.0,2,1.5667,0.0,--|unsecure doublet in noisy region + no signa...,3,none
120,35185760076300581,GALAXY,1.5667,TWL,3,--,--,--,Possible [OII] next to sky lines,1.5667,GALAXY,2.0,2,1.5667,0.0,--|unsecure doublet in noisy region + no signa...,3,none


In [29]:
#copy this text to a new cell to resolve the conflict
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 2
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'flag disagreement, possible weak doublet on skyline, not secure, set to mean of 2'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)

,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
20,35185760076300581,-1,0.33.0.dev3890,0,GALAXY,1.5667,DRP,2,--,--,...,11.692591,1.5667,GALAXY,2.0,2,0.0,1.5667,--|unsecure doublet in noisy region + no signa...,3,"flag disagreement, possible weak doublet on sk..."
70,35185760076300581,-1,0.33.0.dev3890,0,GALAXY,1.5667,vrk,2,--,--,...,11.692591,1.5667,GALAXY,2.0,2,0.0,1.5667,--|unsecure doublet in noisy region + no signa...,3,"flag disagreement, possible weak doublet on sk..."
120,35185760076300581,-1,0.33.0.dev3890,0,GALAXY,1.5667,TWL,2,--,--,...,11.692591,1.5667,GALAXY,2.0,2,0.0,1.5667,--|unsecure doublet in noisy region + no signa...,3,"flag disagreement, possible weak doublet on sk..."


conflict ID =  1


In [30]:
#copy this text to a new cell to display the conflict
conflict_id=2
display_conflict(conflict_id, show_spectra=False)

,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
28,35185760076302567,GALAXY,1.0814,DRP,2,--,--,--,--,1.0814,GALAXY,3.333333,2,1.0814,0.0,--|--|--,3,none
78,35185760076302567,GALAXY,1.0814,vrk,4,--,--,--,--,1.0814,GALAXY,3.333333,2,1.0814,0.0,--|--|--,3,none
128,35185760076302567,GALAXY,1.0814,TWL,4,--,--,--,--,1.0814,GALAXY,3.333333,2,1.0814,0.0,--|--|--,3,none


In [31]:
#copy this text to a new cell to resolve the conflict
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 3
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'flag disagreement but doublet is secure.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)

,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
28,35185760076302567,-1,0.33.0.dev3890,0,GALAXY,1.0814,DRP,3,--,--,...,86.608228,1.0814,GALAXY,3.333333,2,0.0,1.0814,--|--|--,3,flag disagreement but doublet is secure.
78,35185760076302567,-1,0.33.0.dev3890,0,GALAXY,1.0814,vrk,3,--,--,...,86.608228,1.0814,GALAXY,3.333333,2,0.0,1.0814,--|--|--,3,flag disagreement but doublet is secure.
128,35185760076302567,-1,0.33.0.dev3890,0,GALAXY,1.0814,TWL,3,--,--,...,86.608228,1.0814,GALAXY,3.333333,2,0.0,1.0814,--|--|--,3,flag disagreement but doublet is secure.


conflict ID =  2


In [32]:
#copy this text to a new cell to display the conflict
conflict_id=3
display_conflict(conflict_id, show_spectra=False)



,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
33,35185760076303437,GALAXY,1.6171,DRP,1,--,--,--,--,1.6171,GALAXY,2.666667,3,1.6171,0.0,--| clear and strong doublet in noisy region t...,3,none
83,35185760076303437,GALAXY,1.6171,vrk,3,--,--,--,clear and strong doublet in noisy region though,1.6171,GALAXY,2.666667,3,1.6171,0.0,--| clear and strong doublet in noisy region t...,3,none
133,35185760076303437,GALAXY,1.6171,TWL,4,--,--,--,--,1.6171,GALAXY,2.666667,3,1.6171,0.0,--| clear and strong doublet in noisy region t...,3,none


In [33]:
#copy this text to a new cell to resolve the conflict
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 3
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in flag but doublet secures z'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)

,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
33,35185760076303437,-1,0.33.0.dev3890,0,GALAXY,1.6171,DRP,3,--,--,...,39.04475,1.6171,GALAXY,2.666667,3,0.0,1.6171,--| clear and strong doublet in noisy region t...,3,disagreement in flag but doublet secures z
83,35185760076303437,-1,0.33.0.dev3890,0,GALAXY,1.6171,vrk,3,--,--,...,39.04475,1.6171,GALAXY,2.666667,3,0.0,1.6171,--| clear and strong doublet in noisy region t...,3,disagreement in flag but doublet secures z
133,35185760076303437,-1,0.33.0.dev3890,0,GALAXY,1.6171,TWL,3,--,--,...,39.04475,1.6171,GALAXY,2.666667,3,0.0,1.6171,--| clear and strong doublet in noisy region t...,3,disagreement in flag but doublet secures z


conflict ID =  3


In [34]:
#copy this text to a new cell to display the conflict
conflict_id=4
display_conflict(conflict_id, show_spectra=False)

,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
34,35185760076303740,GALAXY,1.2503,DRP,2,--,--,--,--,1.2503,GALAXY,3.333333,2,1.2503,0.0,--|--|--,3,none
84,35185760076303740,GALAXY,1.2503,vrk,4,--,--,--,--,1.2503,GALAXY,3.333333,2,1.2503,0.0,--|--|--,3,none
134,35185760076303740,GALAXY,1.2503,TWL,4,--,--,--,--,1.2503,GALAXY,3.333333,2,1.2503,0.0,--|--|--,3,none


In [35]:
#copy this text to a new cell to resolve the conflict
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 4
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'flag disagreement but doublet secures z'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
34,35185760076303740,-1,0.33.0.dev3890,0,GALAXY,1.2503,DRP,4,--,--,...,112.043267,1.2503,GALAXY,3.333333,2,0.0,1.2503,--|--|--,3,flag disagreement but doublet secures z
84,35185760076303740,-1,0.33.0.dev3890,0,GALAXY,1.2503,vrk,4,--,--,...,112.043267,1.2503,GALAXY,3.333333,2,0.0,1.2503,--|--|--,3,flag disagreement but doublet secures z
134,35185760076303740,-1,0.33.0.dev3890,0,GALAXY,1.2503,TWL,4,--,--,...,112.043267,1.2503,GALAXY,3.333333,2,0.0,1.2503,--|--|--,3,flag disagreement but doublet secures z


conflict ID =  4


In [36]:
#copy this text to a new cell to display the conflict
conflict_id=5
display_conflict(conflict_id, show_spectra=False)

,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
40,35185760076304803,GALAXY,0.0091,DRP,2,--,--,--,--,0.0091,GALAXY,2.333333,1,0.0091,1.068477,--| dubious fit (doublet and lines) + VI redsh...,3,none
90,35185760076304803,GALAXY,0.0091,vrk,2,R,1.0873,--,dubious fit (doublet and lines) + VI redshift...,1.0873,GALAXY,2.333333,1,0.0091,1.068477,--| dubious fit (doublet and lines) + VI redsh...,3,none
140,35185760076304803,GALAXY,0.0091,TWL,3,--,1.0870,--,Possible [OII] at z=1.0870,1.0870,GALAXY,2.333333,1,0.0091,1.068477,--| dubious fit (doublet and lines) + VI redsh...,3,none


In [37]:
#copy this text to a new cell to resolve the conflict
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 1.0871
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 2.5
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in flag and z. 2/3 inspectors suggest z based on OII with mean flag 2.5.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
40,35185760076304803,-1,0.33.0.dev3890,0,GALAXY,0.0091,DRP,2.5,--,--,...,19.483013,1.0871,GALAXY,2.333333,1,1.068477,0.0091,--| dubious fit (doublet and lines) + VI redsh...,3,disagreement in flag and z. 2/3 inspectors sug...
90,35185760076304803,-1,0.33.0.dev3890,0,GALAXY,0.0091,vrk,2.5,R,1.0873,...,19.483013,1.0871,GALAXY,2.333333,1,1.068477,0.0091,--| dubious fit (doublet and lines) + VI redsh...,3,disagreement in flag and z. 2/3 inspectors sug...
140,35185760076304803,-1,0.33.0.dev3890,0,GALAXY,0.0091,TWL,2.5,--,1.0870,...,19.483013,1.0871,GALAXY,2.333333,1,1.068477,0.0091,--| dubious fit (doublet and lines) + VI redsh...,3,disagreement in flag and z. 2/3 inspectors sug...


conflict ID =  5


In [38]:
#copy this text to a new cell to display the conflict
conflict_id=6
display_conflict(conflict_id, show_spectra=False)

,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
45,35185760076305772,GALAXY,0.009,DRP,2.0,--,--,--,--,0.0090,GALAXY,3.0,2,0.009,0.687611,--|dubious fit + VI redshift gives resolved do...,3,none
95,35185760076305772,GALAXY,0.009,vrk,3.0,R,0.7028,--,dubious fit + VI redshift gives resolved doubl...,0.7028,GALAXY,3.0,2,0.009,0.687611,--|dubious fit + VI redshift gives resolved do...,3,none
145,35185760076305772,GALAXY,0.009,TWL,4.0,R,0.7027,--,--,0.7027,GALAXY,3.0,2,0.009,0.687611,--|dubious fit + VI redshift gives resolved do...,3,none


In [39]:
#copy this text to a new cell to resolve the conflict
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 3.5
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in z and flag. 2/3 of inspectors give VI z based on doublet. secure.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
45,35185760076305772,-1,0.33.0.dev3890,0,GALAXY,0.009,DRP,3.5,--,--,...,323.774934,0.0090,GALAXY,3.0,2,0.687611,0.009,--|dubious fit + VI redshift gives resolved do...,3,disagreement in z and flag. 2/3 of inspectors ...
95,35185760076305772,-1,0.33.0.dev3890,0,GALAXY,0.009,vrk,3.5,R,0.7028,...,323.774934,0.7028,GALAXY,3.0,2,0.687611,0.009,--|dubious fit + VI redshift gives resolved do...,3,disagreement in z and flag. 2/3 of inspectors ...
145,35185760076305772,-1,0.33.0.dev3890,0,GALAXY,0.009,TWL,3.5,R,0.7027,...,323.774934,0.7027,GALAXY,3.0,2,0.687611,0.009,--|dubious fit + VI redshift gives resolved do...,3,disagreement in z and flag. 2/3 of inspectors ...


conflict ID =  6


In [40]:
#copy this text to a new cell to display the conflict
conflict_id=7
display_conflict(conflict_id, show_spectra=False)


,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
1,35185760080494926,GALAXY,1.6258,DRP,0.0,--,--,--,--,1.6258,GALAXY,1.0,3,1.626267,0.000533,--|doublet with one very strong line at the ed...,3,none
51,35185760080494926,GALAXY,1.6258,vrk,3.0,--,1.6272,--,doublet with one very strong line at the edge ...,1.6272,GALAXY,1.0,3,1.626267,0.000533,--|doublet with one very strong line at the ed...,3,none
101,35185760080494926,GALAXY,1.6258,TWL,0.0,--,--,--,Low S/N,1.6258,GALAXY,1.0,3,1.626267,0.000533,--|doublet with one very strong line at the ed...,3,none


In [41]:
#copy this text to a new cell to resolve the conflict
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 1
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in flag. z not secure. set to mean flag of 1.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
1,35185760080494926,-1,0.33.0.dev3890,0,GALAXY,1.6258,DRP,1.0,--,--,...,4.442083,1.6258,GALAXY,1.0,3,0.000533,1.626267,--|doublet with one very strong line at the ed...,3,disagreement in flag. z not secure. set to mea...
51,35185760080494926,-1,0.33.0.dev3890,0,GALAXY,1.6258,vrk,1.0,--,1.6272,...,4.442083,1.6272,GALAXY,1.0,3,0.000533,1.626267,--|doublet with one very strong line at the ed...,3,disagreement in flag. z not secure. set to mea...
101,35185760080494926,-1,0.33.0.dev3890,0,GALAXY,1.6258,TWL,1.0,--,--,...,4.442083,1.6258,GALAXY,1.0,3,0.000533,1.626267,--|doublet with one very strong line at the ed...,3,disagreement in flag. z not secure. set to mea...


conflict ID =  7


In [42]:
#copy this text to a new cell to display the conflict
conflict_id=8
display_conflict(conflict_id, show_spectra=False)

,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
3,35185760080495716,GALAXY,1.5493,DRP,1.0,--,--,--,--,1.5493,GALAXY,2.333333,2,1.5493,0.0,--| resolved doublet in noisy region though| [...,3,none
53,35185760080495716,GALAXY,1.5493,vrk,3.0,--,--,--,resolved doublet in noisy region though,1.5493,GALAXY,2.333333,2,1.5493,0.0,--| resolved doublet in noisy region though| [...,3,none
103,35185760080495716,GALAXY,1.5493,TWL,3.0,--,--,--,[OII] emission next to sky lines,1.5493,GALAXY,2.333333,2,1.5493,0.0,--| resolved doublet in noisy region though| [...,3,none


In [43]:
#copy this text to a new cell to resolve the conflict
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 3
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'flag disagreement but doublet secures z.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)



,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
3,35185760080495716,-1,0.33.0.dev3890,0,GALAXY,1.5493,DRP,3.0,--,--,...,62.26452,1.5493,GALAXY,2.333333,2,0.0,1.5493,--| resolved doublet in noisy region though| [...,3,flag disagreement but doublet secures z.
53,35185760080495716,-1,0.33.0.dev3890,0,GALAXY,1.5493,vrk,3.0,--,--,...,62.26452,1.5493,GALAXY,2.333333,2,0.0,1.5493,--| resolved doublet in noisy region though| [...,3,flag disagreement but doublet secures z.
103,35185760080495716,-1,0.33.0.dev3890,0,GALAXY,1.5493,TWL,3.0,--,--,...,62.26452,1.5493,GALAXY,2.333333,2,0.0,1.5493,--| resolved doublet in noisy region though| [...,3,flag disagreement but doublet secures z.


conflict ID =  8


In [44]:
#copy this text to a new cell to display the conflict
conflict_id=9
display_conflict(conflict_id, show_spectra=False)


,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
162,35185760084690463,GALAXY,1.1655,TWL,4.0,--,--,--,Broad [OII],1.1655,GALAXY,3.333333,2,1.1655,0.0,Broad [OII]| possible OII|--,3,none
212,35185760084690463,GALAXY,1.1655,SEM,2.0,--,--,--,possible OII,1.1655,GALAXY,3.333333,2,1.1655,0.0,Broad [OII]| possible OII|--,3,none
262,35185760084690463,GALAXY,1.1655,VRK,4.0,--,--,--,--,1.1655,GALAXY,3.333333,2,1.1655,0.0,Broad [OII]| possible OII|--,3,none


In [45]:
#copy this text to a new cell to resolve the conflict
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 3
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in flag but z is secure - broad OII and continuum'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
162,35185760084690463,-1,0.33.0.dev3890,0,GALAXY,1.1655,TWL,3.0,--,--,...,194.289187,1.1655,GALAXY,3.333333,2,0.0,1.1655,Broad [OII]| possible OII|--,3,disagreement in flag but z is secure - broad O...
212,35185760084690463,-1,0.33.0.dev3890,0,GALAXY,1.1655,SEM,3.0,--,--,...,194.289187,1.1655,GALAXY,3.333333,2,0.0,1.1655,Broad [OII]| possible OII|--,3,disagreement in flag but z is secure - broad O...
262,35185760084690463,-1,0.33.0.dev3890,0,GALAXY,1.1655,VRK,3.0,--,--,...,194.289187,1.1655,GALAXY,3.333333,2,0.0,1.1655,Broad [OII]| possible OII|--,3,disagreement in flag but z is secure - broad O...


conflict ID =  9


In [46]:
#copy this text to a new cell to display the conflict
conflict_id=10
display_conflict(conflict_id, show_spectra=False)


,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
169,35185760084692144,GALAXY,1.0185,TWL,0.0,--,--,--,Low S/N,1.0185,GALAXY,1.0,3,1.0185,0.0,Low S/N| OII| dubious dfit (unconvincing doub...,3,none
219,35185760084692144,GALAXY,1.0185,SEM,3.0,--,--,--,OII,1.0185,GALAXY,1.0,3,1.0185,0.0,Low S/N| OII| dubious dfit (unconvincing doub...,3,none
269,35185760084692144,GALAXY,1.0185,VRK,0.0,R,--,--,dubious dfit (unconvincing doublet + all othe...,1.0185,GALAXY,1.0,3,1.0185,0.0,Low S/N| OII| dubious dfit (unconvincing doub...,3,none


In [47]:
#copy this text to a new cell to resolve the conflict
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 1
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'flag disagreement. z not secure, set to mean of 1.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
169,35185760084692144,-1,0.33.0.dev3890,0,GALAXY,1.0185,TWL,1.0,--,--,...,0.853249,1.0185,GALAXY,1.0,3,0.0,1.0185,Low S/N| OII| dubious dfit (unconvincing doub...,3,"flag disagreement. z not secure, set to mean o..."
219,35185760084692144,-1,0.33.0.dev3890,0,GALAXY,1.0185,SEM,1.0,--,--,...,0.853249,1.0185,GALAXY,1.0,3,0.0,1.0185,Low S/N| OII| dubious dfit (unconvincing doub...,3,"flag disagreement. z not secure, set to mean o..."
269,35185760084692144,-1,0.33.0.dev3890,0,GALAXY,1.0185,VRK,1.0,R,--,...,0.853249,1.0185,GALAXY,1.0,3,0.0,1.0185,Low S/N| OII| dubious dfit (unconvincing doub...,3,"flag disagreement. z not secure, set to mean o..."


conflict ID =  10


In [48]:
#copy this text to a new cell to display the conflict
conflict_id=11
display_conflict(conflict_id, show_spectra=False)


,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
184,35185760084692854,GALAXY,0.7714,TWL,0.0,--,--,--,Low S/N,0.7714,GALAXY,1.333333,3,0.7714,0.0,Low S/N| low SNR|--,3,none
234,35185760084692854,GALAXY,0.7714,SEM,1.0,--,--,--,low SNR,0.7714,GALAXY,1.333333,3,0.7714,0.0,Low S/N| low SNR|--,3,none
284,35185760084692854,GALAXY,0.7714,VRK,3.0,--,--,--,--,0.7714,GALAXY,1.333333,3,0.7714,0.0,Low S/N| low SNR|--,3,none


In [49]:
#copy this text to a new cell to resolve the conflict
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 1.333333
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in flag. z not secure, set to mean of 1.3'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
184,35185760084692854,-1,0.33.0.dev3890,0,GALAXY,0.7714,TWL,1.333333,--,--,...,2.612015,0.7714,GALAXY,1.333333,3,0.0,0.7714,Low S/N| low SNR|--,3,"disagreement in flag. z not secure, set to mea..."
234,35185760084692854,-1,0.33.0.dev3890,0,GALAXY,0.7714,SEM,1.333333,--,--,...,2.612015,0.7714,GALAXY,1.333333,3,0.0,0.7714,Low S/N| low SNR|--,3,"disagreement in flag. z not secure, set to mea..."
284,35185760084692854,-1,0.33.0.dev3890,0,GALAXY,0.7714,VRK,1.333333,--,--,...,2.612015,0.7714,GALAXY,1.333333,3,0.0,0.7714,Low S/N| low SNR|--,3,"disagreement in flag. z not secure, set to mea..."


conflict ID =  11


In [50]:
#copy this text to a new cell to display the conflict
conflict_id=12
display_conflict(conflict_id, show_spectra=False)

,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
332,35185760093078527,QSO,1.412,TWL,1.0,--,--,--,best fit [OII] lines in skyline region,1.4120,QSO,2.0,2,1.412,1.4993,best fit [OII] lines in skyline region|--| No...,3,none
382,35185760093078527,QSO,1.412,SEM,2.0,R,1.4993,--,--,1.4993,QSO,2.0,2,1.412,1.4993,best fit [OII] lines in skyline region|--| No...,3,none
432,35185760093078527,QSO,1.412,PNT,3.0,RCS,0,--,Noisy spectra,0.0000,QSO,2.0,2,1.412,1.4993,best fit [OII] lines in skyline region|--| No...,3,none


In [51]:
#copy this text to a new cell to resolve the conflict
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 1.412
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 1
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in z and flag. no signal, set to redrock solution flag 1'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
332,35185760093078527,-1,0.33.0.dev3890,0,QSO,1.412,TWL,1.0,--,--,...,18.658793,1.412,QSO,2.0,2,1.4993,1.412,best fit [OII] lines in skyline region|--| No...,3,"disagreement in z and flag. no signal, set to ..."
382,35185760093078527,-1,0.33.0.dev3890,0,QSO,1.412,SEM,1.0,R,1.4993,...,18.658793,1.412,QSO,2.0,2,1.4993,1.412,best fit [OII] lines in skyline region|--| No...,3,"disagreement in z and flag. no signal, set to ..."
432,35185760093078527,-1,0.33.0.dev3890,0,QSO,1.412,PNT,1.0,RCS,0,...,18.658793,1.412,QSO,2.0,2,1.4993,1.412,best fit [OII] lines in skyline region|--| No...,3,"disagreement in z and flag. no signal, set to ..."


conflict ID =  12


In [52]:
#copy this text to a new cell to display the conflict
conflict_id=13
display_conflict(conflict_id, show_spectra=False)


,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
338,35185760093081089,GALAXY,1.1455,TWL,4.0,--,--,--,--,1.1455,GALAXY,2.666667,2,1.1455,0.0,--| OII|--,3,none
388,35185760093081089,GALAXY,1.1455,SEM,2.0,--,--,--,OII,1.1455,GALAXY,2.666667,2,1.1455,0.0,--| OII|--,3,none
438,35185760093081089,GALAXY,1.1455,PNT,2.0,--,1.1455,GALAXY,--,1.1455,GALAXY,2.666667,2,1.1455,0.0,--| OII|--,3,none


In [53]:
#copy this text to a new cell to resolve the conflict
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 2.666667
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in flag. set to mean.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)



,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
338,35185760093081089,-1,0.33.0.dev3890,0,GALAXY,1.1455,TWL,2.666667,--,--,...,351.937334,1.1455,GALAXY,2.666667,2,0.0,1.1455,--| OII|--,3,disagreement in flag. set to mean.
388,35185760093081089,-1,0.33.0.dev3890,0,GALAXY,1.1455,SEM,2.666667,--,--,...,351.937334,1.1455,GALAXY,2.666667,2,0.0,1.1455,--| OII|--,3,disagreement in flag. set to mean.
438,35185760093081089,-1,0.33.0.dev3890,0,GALAXY,1.1455,PNT,2.666667,--,1.1455,...,351.937334,1.1455,GALAXY,2.666667,2,0.0,1.1455,--| OII|--,3,disagreement in flag. set to mean.


conflict ID =  13


In [54]:
#copy this text to a new cell to display the conflict
conflict_id=14
display_conflict(conflict_id, show_spectra=False)

,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
339,35185760093081557,GALAXY,0.9444,TWL,4.0,--,--,--,AGN with broad MgII emission which is not cap...,0.9444,GALAXY,3.333333,2,0.9444,0.0,AGN with broad MgII emission which is not cap...,3,none
389,35185760093081557,GALAXY,0.9444,SEM,4.0,--,--,--,OII and OIII,0.9444,GALAXY,3.333333,2,0.9444,0.0,AGN with broad MgII emission which is not cap...,3,none
439,35185760093081557,GALAXY,0.9444,PNT,2.0,--,--,GALAXY,OII and OIII lines are resolved but there is ...,0.9444,GALAXY,3.333333,2,0.9444,0.0,AGN with broad MgII emission which is not cap...,3,none


In [55]:
#copy this text to a new cell to resolve the conflict
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 4
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 'QSO'

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in flag but z is secure. broad MgII emission, set spectype to QSO'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
339,35185760093081557,-1,0.33.0.dev3890,0,GALAXY,0.9444,TWL,4.0,--,--,...,311.324931,0.9444,QSO,3.333333,2,0.0,0.9444,AGN with broad MgII emission which is not cap...,3,disagreement in flag but z is secure. broad Mg...
389,35185760093081557,-1,0.33.0.dev3890,0,GALAXY,0.9444,SEM,4.0,--,--,...,311.324931,0.9444,QSO,3.333333,2,0.0,0.9444,AGN with broad MgII emission which is not cap...,3,disagreement in flag but z is secure. broad Mg...
439,35185760093081557,-1,0.33.0.dev3890,0,GALAXY,0.9444,PNT,4.0,--,--,...,311.324931,0.9444,QSO,3.333333,2,0.0,0.9444,AGN with broad MgII emission which is not cap...,3,disagreement in flag but z is secure. broad Mg...


conflict ID =  14


In [56]:
#copy this text to a new cell to display the conflict
conflict_id=15
display_conflict(conflict_id, show_spectra=False)


,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
341,35185760097272055,GALAXY,0.6616,TWL,4.0,--,--,--,--,0.6616,GALAXY,2.333333,3,0.6616,0.0,--| OII| Noisy spectra no line is resolved onl...,3,none
391,35185760097272055,GALAXY,0.6616,SEM,2.0,--,--,--,OII,0.6616,GALAXY,2.333333,3,0.6616,0.0,--| OII| Noisy spectra no line is resolved onl...,3,none
441,35185760097272055,GALAXY,0.6616,PNT,1.0,S,0.6616,GALAXY,Noisy spectra no line is resolved only contin...,0.6616,GALAXY,2.333333,3,0.6616,0.0,--| OII| Noisy spectra no line is resolved onl...,3,none


In [57]:
#copy this text to a new cell to resolve the conflict
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 3
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in flag, but z is secure, oII and balmer lines'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)

,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
341,35185760097272055,-1,0.33.0.dev3890,0,GALAXY,0.6616,TWL,3.0,--,--,...,101.799279,0.6616,GALAXY,2.333333,3,0.0,0.6616,--| OII| Noisy spectra no line is resolved onl...,3,"disagreement in flag, but z is secure, oII and..."
391,35185760097272055,-1,0.33.0.dev3890,0,GALAXY,0.6616,SEM,3.0,--,--,...,101.799279,0.6616,GALAXY,2.333333,3,0.0,0.6616,--| OII| Noisy spectra no line is resolved onl...,3,"disagreement in flag, but z is secure, oII and..."
441,35185760097272055,-1,0.33.0.dev3890,0,GALAXY,0.6616,PNT,3.0,S,0.6616,...,101.799279,0.6616,GALAXY,2.333333,3,0.0,0.6616,--| OII| Noisy spectra no line is resolved onl...,3,"disagreement in flag, but z is secure, oII and..."


conflict ID =  15


In [58]:
#copy this text to a new cell to display the conflict
conflict_id=16
display_conflict(conflict_id, show_spectra=False)

,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
344,35185760097272551,GALAXY,0.8696,TWL,2.0,--,--,--,--,0.8696,GALAXY,3.0,2,0.8696,0.0,--| OII and balmer lines|--,3,none
394,35185760097272551,GALAXY,0.8696,SEM,4.0,--,--,--,OII and balmer lines,0.8696,GALAXY,3.0,2,0.8696,0.0,--| OII and balmer lines|--,3,none
443,35185760097272551,GALAXY,0.8696,PNT,3.0,--,0.8696,GALAXY,--,0.8696,GALAXY,3.0,2,0.8696,0.0,--| OII and balmer lines|--,3,none


In [59]:
#copy this text to a new cell to resolve the conflict
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 3
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'flag disagreement, take mean.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)

,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
344,35185760097272551,-1,0.33.0.dev3890,0,GALAXY,0.8696,TWL,3.0,--,--,...,47.603317,0.8696,GALAXY,3.0,2,0.0,0.8696,--| OII and balmer lines|--,3,"flag disagreement, take mean."
394,35185760097272551,-1,0.33.0.dev3890,0,GALAXY,0.8696,SEM,3.0,--,--,...,47.603317,0.8696,GALAXY,3.0,2,0.0,0.8696,--| OII and balmer lines|--,3,"flag disagreement, take mean."
443,35185760097272551,-1,0.33.0.dev3890,0,GALAXY,0.8696,PNT,3.0,--,0.8696,...,47.603317,0.8696,GALAXY,3.0,2,0.0,0.8696,--| OII and balmer lines|--,3,"flag disagreement, take mean."


conflict ID =  16


In [60]:
#copy this text to a new cell to display the conflict
conflict_id=17
display_conflict(conflict_id, show_spectra=False)


,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
303,35185760097273421,GALAXY,0.1197,TWL,2.0,--,--,--,Low S/N,0.1197,GALAXY,2.666667,2,0.1197,0.1197,Low S/N| OIII and Ha| Noisy spectra,3,none
353,35185760097273421,GALAXY,0.1197,SEM,4.0,--,--,--,OIII and Ha,0.1197,GALAXY,2.666667,2,0.1197,0.1197,Low S/N| OIII and Ha| Noisy spectra,3,none
403,35185760097273421,GALAXY,0.1197,PNT,2.0,RCS,0,--,Noisy spectra,0.0000,GALAXY,2.666667,2,0.1197,0.1197,Low S/N| OIII and Ha| Noisy spectra,3,none


In [61]:
#copy this text to a new cell to resolve the conflict
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 2
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in flag. z not secure, set to 2.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)

,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
303,35185760097273421,-1,0.33.0.dev3890,0,GALAXY,0.1197,TWL,2.0,--,--,...,1.087889,0.1197,GALAXY,2.666667,2,0.1197,0.1197,Low S/N| OIII and Ha| Noisy spectra,3,"disagreement in flag. z not secure, set to 2."
353,35185760097273421,-1,0.33.0.dev3890,0,GALAXY,0.1197,SEM,2.0,--,--,...,1.087889,0.1197,GALAXY,2.666667,2,0.1197,0.1197,Low S/N| OIII and Ha| Noisy spectra,3,"disagreement in flag. z not secure, set to 2."
403,35185760097273421,-1,0.33.0.dev3890,0,GALAXY,0.1197,PNT,2.0,RCS,0,...,1.087889,0.0000,GALAXY,2.666667,2,0.1197,0.1197,Low S/N| OIII and Ha| Noisy spectra,3,"disagreement in flag. z not secure, set to 2."


conflict ID =  17


In [62]:
#copy this text to a new cell to display the conflict
conflict_id=18
display_conflict(conflict_id, show_spectra=False)

,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
349,35185760097274623,GALAXY,1.4023,TWL,1.0,R,--,--,Best fit on a strong sky line residual,1.4023,GALAXY,2.0,2,1.4023,0.0,Best fit on a strong sky line residual| possi...,3,none
399,35185760097274623,GALAXY,1.4023,SEM,3.0,--,--,--,possible OII doublet and weak balmer lines,1.4023,GALAXY,2.0,2,1.4023,0.0,Best fit on a strong sky line residual| possi...,3,none
448,35185760097274623,GALAXY,1.4023,PNT,2.0,S,1.4023,GALAXY,Noisy spectra only one of the doublets of OII...,1.4023,GALAXY,2.0,2,1.4023,0.0,Best fit on a strong sky line residual| possi...,3,none


In [63]:
#copy this text to a new cell to resolve the conflict
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 2
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in flag but z not secure. set to mean flag of 2'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
349,35185760097274623,-1,0.33.0.dev3890,0,GALAXY,1.4023,TWL,2.0,R,--,...,8.397231,1.4023,GALAXY,2.0,2,0.0,1.4023,Best fit on a strong sky line residual| possi...,3,disagreement in flag but z not secure. set to ...
399,35185760097274623,-1,0.33.0.dev3890,0,GALAXY,1.4023,SEM,2.0,--,--,...,8.397231,1.4023,GALAXY,2.0,2,0.0,1.4023,Best fit on a strong sky line residual| possi...,3,disagreement in flag but z not secure. set to ...
448,35185760097274623,-1,0.33.0.dev3890,0,GALAXY,1.4023,PNT,2.0,S,1.4023,...,8.397231,1.4023,GALAXY,2.0,2,0.0,1.4023,Best fit on a strong sky line residual| possi...,3,disagreement in flag but z not secure. set to ...


conflict ID =  18


In [64]:
#copy this text to a new cell to display the conflict
conflict_id=19
display_conflict(conflict_id, show_spectra=False)



,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
306,35185760097274678,QSO,1.1094,TWL,1.0,--,--,--,--,1.1094,QSO,2.0,2,1.1094,0.0,--| OII| Noisy spectra nothing resloved,3,none
356,35185760097274678,QSO,1.1094,SEM,2.0,--,--,--,OII,1.1094,QSO,2.0,2,1.1094,0.0,--| OII| Noisy spectra nothing resloved,3,none
406,35185760097274678,QSO,1.1094,PNT,3.0,RCS,--,--,Noisy spectra nothing resloved,1.1094,QSO,2.0,2,1.1094,0.0,--| OII| Noisy spectra nothing resloved,3,none


In [65]:
#copy this text to a new cell to resolve the conflict
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 2
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'flag disagreement but z is not secure. set to mean flag of 2.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)

,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
306,35185760097274678,-1,0.33.0.dev3890,0,QSO,1.1094,TWL,2.0,--,--,...,2.618959,1.1094,QSO,2.0,2,0.0,1.1094,--| OII| Noisy spectra nothing resloved,3,flag disagreement but z is not secure. set to ...
356,35185760097274678,-1,0.33.0.dev3890,0,QSO,1.1094,SEM,2.0,--,--,...,2.618959,1.1094,QSO,2.0,2,0.0,1.1094,--| OII| Noisy spectra nothing resloved,3,flag disagreement but z is not secure. set to ...
406,35185760097274678,-1,0.33.0.dev3890,0,QSO,1.1094,PNT,2.0,RCS,--,...,2.618959,1.1094,QSO,2.0,2,0.0,1.1094,--| OII| Noisy spectra nothing resloved,3,flag disagreement but z is not secure. set to ...


conflict ID =  19


In [66]:
#copy this text to a new cell to display the conflict
conflict_id=20
display_conflict(conflict_id, show_spectra=False)

,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
307,35185760097275227,GALAXY,1.3708,TWL,2.0,--,--,--,possible [OII] in skyline region,1.3708,GALAXY,2.333333,1,1.3708,1.3708,possible [OII] in skyline region| OII| Noisy ...,3,none
357,35185760097275227,GALAXY,1.3708,SEM,2.0,--,--,--,OII,1.3708,GALAXY,2.333333,1,1.3708,1.3708,possible [OII] in skyline region| OII| Noisy ...,3,none
407,35185760097275227,GALAXY,1.3708,PNT,3.0,--,0,GALAXY,Noisy spectra,0.0000,GALAXY,2.333333,1,1.3708,1.3708,possible [OII] in skyline region| OII| Noisy ...,3,none


In [67]:
#copy this text to a new cell to resolve the conflict
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 1.3708
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in z, one inspector set z=0'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
307,35185760097275227,-1,0.33.0.dev3890,0,GALAXY,1.3708,TWL,2.0,--,--,...,40.644888,1.3708,GALAXY,2.333333,1,1.3708,1.3708,possible [OII] in skyline region| OII| Noisy ...,3,"disagreement in z, one inspector set z=0"
357,35185760097275227,-1,0.33.0.dev3890,0,GALAXY,1.3708,SEM,2.0,--,--,...,40.644888,1.3708,GALAXY,2.333333,1,1.3708,1.3708,possible [OII] in skyline region| OII| Noisy ...,3,"disagreement in z, one inspector set z=0"
407,35185760097275227,-1,0.33.0.dev3890,0,GALAXY,1.3708,PNT,3.0,--,0,...,40.644888,1.3708,GALAXY,2.333333,1,1.3708,1.3708,possible [OII] in skyline region| OII| Noisy ...,3,"disagreement in z, one inspector set z=0"


conflict ID =  20


In [68]:
#copy this text to a new cell to display the conflict
conflict_id=21
display_conflict(conflict_id, show_spectra=False)


,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
311,35185760097275715,GALAXY,1.6366,TWL,0.0,RC,--,--,Very low S/N,1.6366,GALAXY,1.333333,3,1.6366,1.6366,Very low S/N| low SNR|Noisy spectra bad class...,3,none
361,35185760097275715,GALAXY,1.6366,SEM,1.0,--,--,--,low SNR,1.6366,GALAXY,1.333333,3,1.6366,1.6366,Very low S/N| low SNR|Noisy spectra bad class...,3,none
411,35185760097275715,GALAXY,1.6366,PNT,3.0,RCS,0,--,Noisy spectra bad classification and redshift,0.0000,GALAXY,1.333333,3,1.6366,1.6366,Very low S/N| low SNR|Noisy spectra bad class...,3,none


In [69]:
#copy this text to a new cell to resolve the conflict
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 1.6366
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 1.333333
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in flag but z not secure. set flag to mean 1.3'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
311,35185760097275715,-1,0.33.0.dev3890,0,GALAXY,1.6366,TWL,1.333333,RC,--,...,1.960007,1.6366,GALAXY,1.333333,3,1.6366,1.6366,Very low S/N| low SNR|Noisy spectra bad class...,3,disagreement in flag but z not secure. set fla...
361,35185760097275715,-1,0.33.0.dev3890,0,GALAXY,1.6366,SEM,1.333333,--,--,...,1.960007,1.6366,GALAXY,1.333333,3,1.6366,1.6366,Very low S/N| low SNR|Noisy spectra bad class...,3,disagreement in flag but z not secure. set fla...
411,35185760097275715,-1,0.33.0.dev3890,0,GALAXY,1.6366,PNT,1.333333,RCS,0,...,1.960007,1.6366,GALAXY,1.333333,3,1.6366,1.6366,Very low S/N| low SNR|Noisy spectra bad class...,3,disagreement in flag but z not secure. set fla...


conflict ID =  21


In [70]:
#copy this text to a new cell to display the conflict
conflict_id=22
display_conflict(conflict_id, show_spectra=False)


,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
313,35185760097276144,GALAXY,1.0135,TWL,1.0,--,--,--,--,1.0135,GALAXY,2.0,3,1.0135,0.013558,--|--| Noisy spectra,3,none
363,35185760097276144,GALAXY,1.0135,SEM,1.0,R,1.0408,--,--,1.0408,GALAXY,2.0,3,1.0135,0.013558,--|--| Noisy spectra,3,none
413,35185760097276144,GALAXY,1.0135,PNT,4.0,RCS,--,--,Noisy spectra,1.0135,GALAXY,2.0,3,1.0135,0.013558,--|--| Noisy spectra,3,none


In [71]:
#copy this text to a new cell to resolve the conflict
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 1.0135
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 2
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in flag and z. set to mean flag 2 and rr z.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
313,35185760097276144,-1,0.33.0.dev3890,0,GALAXY,1.0135,TWL,2.0,--,--,...,18.247463,1.0135,GALAXY,2.0,3,0.013558,1.0135,--|--| Noisy spectra,3,disagreement in flag and z. set to mean flag 2...
363,35185760097276144,-1,0.33.0.dev3890,0,GALAXY,1.0135,SEM,2.0,R,1.0408,...,18.247463,1.0135,GALAXY,2.0,3,0.013558,1.0135,--|--| Noisy spectra,3,disagreement in flag and z. set to mean flag 2...
413,35185760097276144,-1,0.33.0.dev3890,0,GALAXY,1.0135,PNT,2.0,RCS,--,...,18.247463,1.0135,GALAXY,2.0,3,0.013558,1.0135,--|--| Noisy spectra,3,disagreement in flag and z. set to mean flag 2...


conflict ID =  22


In [72]:
#copy this text to a new cell to display the conflict
conflict_id=23
display_conflict(conflict_id, show_spectra=False)

,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
326,35185760101466913,GALAXY,0.0386,TWL,4.0,--,--,--,--,0.0386,GALAXY,3.333333,2,0.0386,0.0,--| OII and OIII and balmer lines|Probably gal...,3,none
376,35185760101466913,GALAXY,0.0386,SEM,4.0,--,--,--,OII and OIII and balmer lines,0.0386,GALAXY,3.333333,2,0.0386,0.0,--| OII and OIII and balmer lines|Probably gal...,3,none
426,35185760101466913,GALAXY,0.0386,PNT,2.0,S,0.0386,GALAXY,Probably galaxy but maybe a star according to ...,0.0386,GALAXY,3.333333,2,0.0386,0.0,--| OII and OIII and balmer lines|Probably gal...,3,none


In [73]:
print(vi.loc[426]['VI comment'])

Probably galaxy but maybe a star according to the fit


In [74]:
#copy this text to a new cell to resolve the conflict
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 4
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in flag but z is secure'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
326,35185760101466913,-1,0.33.0.dev3890,0,GALAXY,0.0386,TWL,4.0,--,--,...,1605.635982,0.0386,GALAXY,3.333333,2,0.0,0.0386,--| OII and OIII and balmer lines|Probably gal...,3,disagreement in flag but z is secure
376,35185760101466913,-1,0.33.0.dev3890,0,GALAXY,0.0386,SEM,4.0,--,--,...,1605.635982,0.0386,GALAXY,3.333333,2,0.0,0.0386,--| OII and OIII and balmer lines|Probably gal...,3,disagreement in flag but z is secure
426,35185760101466913,-1,0.33.0.dev3890,0,GALAXY,0.0386,PNT,4.0,S,0.0386,...,1605.635982,0.0386,GALAXY,3.333333,2,0.0,0.0386,--| OII and OIII and balmer lines|Probably gal...,3,disagreement in flag but z is secure


conflict ID =  23


In [75]:
#copy this text to a new cell to display the conflict
conflict_id=24
display_conflict(conflict_id, show_spectra=False)

,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
328,35185760101467485,GALAXY,1.4753,TWL,4.0,--,--,--,--,1.4753,GALAXY,3.0,2,1.4753,0.0,--| possible OII doublet| OII,3,none
378,35185760101467485,GALAXY,1.4753,SEM,2.0,--,--,--,possible OII doublet,1.4753,GALAXY,3.0,2,1.4753,0.0,--| possible OII doublet| OII,3,none
428,35185760101467485,GALAXY,1.4753,PNT,3.0,--,1.4753,GALAXY,OII,1.4753,GALAXY,3.0,2,1.4753,0.0,--| possible OII doublet| OII,3,none


In [76]:
#copy this text to a new cell to resolve the conflict
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 3
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in flag but z is secure'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)

,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
328,35185760101467485,-1,0.33.0.dev3890,0,GALAXY,1.4753,TWL,3.0,--,--,...,242.447919,1.4753,GALAXY,3.0,2,0.0,1.4753,--| possible OII doublet| OII,3,disagreement in flag but z is secure
378,35185760101467485,-1,0.33.0.dev3890,0,GALAXY,1.4753,SEM,3.0,--,--,...,242.447919,1.4753,GALAXY,3.0,2,0.0,1.4753,--| possible OII doublet| OII,3,disagreement in flag but z is secure
428,35185760101467485,-1,0.33.0.dev3890,0,GALAXY,1.4753,PNT,3.0,--,1.4753,...,242.447919,1.4753,GALAXY,3.0,2,0.0,1.4753,--| possible OII doublet| OII,3,disagreement in flag but z is secure


conflict ID =  24


In [77]:
#copy this text to a new cell to display the conflict
conflict_id=25
display_conflict(conflict_id, show_spectra=False)

,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
331,35185760101467682,GALAXY,1.2101,TWL,0.0,--,--,--,too low S/N,1.2101,GALAXY,1.333333,3,1.2101,0.0,too low S/N|--| OII line is resolved with sma...,3,none
381,35185760101467682,GALAXY,1.2101,SEM,1.0,R,--,--,--,1.2101,GALAXY,1.333333,3,1.2101,0.0,too low S/N|--| OII line is resolved with sma...,3,none
431,35185760101467682,GALAXY,1.2101,PNT,3.0,--,1.2101,GALAXY,OII line is resolved with small signal to noi...,1.2101,GALAXY,1.333333,3,1.2101,0.0,too low S/N|--| OII line is resolved with sma...,3,none


In [78]:
#copy this text to a new cell to resolve the conflict
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 1
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in flag. z not secure. flag 1.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)

,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
331,35185760101467682,-1,0.33.0.dev3890,0,GALAXY,1.2101,TWL,1.0,--,--,...,0.71738,1.2101,GALAXY,1.333333,3,0.0,1.2101,too low S/N|--| OII line is resolved with sma...,3,disagreement in flag. z not secure. flag 1.
381,35185760101467682,-1,0.33.0.dev3890,0,GALAXY,1.2101,SEM,1.0,R,--,...,0.71738,1.2101,GALAXY,1.333333,3,0.0,1.2101,too low S/N|--| OII line is resolved with sma...,3,disagreement in flag. z not secure. flag 1.
431,35185760101467682,-1,0.33.0.dev3890,0,GALAXY,1.2101,PNT,1.0,--,1.2101,...,0.71738,1.2101,GALAXY,1.333333,3,0.0,1.2101,too low S/N|--| OII line is resolved with sma...,3,disagreement in flag. z not secure. flag 1.


conflict ID =  25


In [79]:
#copy this text to a new cell to display the conflict
conflict_id=26
display_conflict(conflict_id, show_spectra=False)



,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
455,35185760101468803,GALAXY,0.8348,PNT,4.0,--,0.8348,GALAXY,OII doublet is not quite resolved,0.8348,GALAXY,3.25,2,0.8348,0.304175,OII doublet is not quite resolved| unconvinci...,4,none
505,35185760101468803,GALAXY,0.8348,VRK,3.0,--,--,--,unconvincing doublet (weak) but Balmer lines ...,0.8348,GALAXY,3.25,2,0.8348,0.304175,OII doublet is not quite resolved| unconvinci...,4,none
555,35185760101468803,GALAXY,0.8348,ACE,2.0,--,1.3929,GALAXY,OII in a sky line.,1.3929,GALAXY,3.25,2,0.8348,0.304175,OII doublet is not quite resolved| unconvinci...,4,none
605,35185760101468803,GALAXY,0.8348,TWL,4.0,--,--,--,--,0.8348,GALAXY,3.25,2,0.8348,0.304175,OII doublet is not quite resolved| unconvinci...,4,none


In [80]:
#copy this text to a new cell to resolve the conflict
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 0.8348
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 3
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in z and flag. take rr z - weak OII but many weak balmer lines in absorption'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)

,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
455,35185760101468803,-1,0.33.0.dev3890,0,GALAXY,0.8348,PNT,3.0,--,0.8348,...,77.534966,0.8348,GALAXY,3.25,2,0.304175,0.8348,OII doublet is not quite resolved| unconvinci...,4,disagreement in z and flag. take rr z - weak O...
505,35185760101468803,-1,0.33.0.dev3890,0,GALAXY,0.8348,VRK,3.0,--,--,...,77.534966,0.8348,GALAXY,3.25,2,0.304175,0.8348,OII doublet is not quite resolved| unconvinci...,4,disagreement in z and flag. take rr z - weak O...
555,35185760101468803,-1,0.33.0.dev3890,0,GALAXY,0.8348,ACE,3.0,--,1.3929,...,77.534966,0.8348,GALAXY,3.25,2,0.304175,0.8348,OII doublet is not quite resolved| unconvinci...,4,disagreement in z and flag. take rr z - weak O...
605,35185760101468803,-1,0.33.0.dev3890,0,GALAXY,0.8348,TWL,3.0,--,--,...,77.534966,0.8348,GALAXY,3.25,2,0.304175,0.8348,OII doublet is not quite resolved| unconvinci...,4,disagreement in z and flag. take rr z - weak O...


conflict ID =  26


In [81]:
#copy this text to a new cell to display the conflict
conflict_id=27
display_conflict(conflict_id, show_spectra=False)

,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
458,35185760101468965,QSO,2.94,PNT,4.0,RCS,0,--,Noisy spectra bad classification and redshift...,0.00,QSO,3.0,2,2.94,2.94,Noisy spectra bad classification and redshift...,4,none
508,35185760101468965,QSO,2.94,VRK,3.0,--,--,--,not an ELG + QSO seems likely,2.94,QSO,3.0,2,2.94,2.94,Noisy spectra bad classification and redshift...,4,none
558,35185760101468965,QSO,2.94,ACE,3.0,--,--,--,--,2.94,QSO,3.0,2,2.94,2.94,Noisy spectra bad classification and redshift...,4,none
608,35185760101468965,QSO,2.94,TWL,2.0,--,--,--,--,2.94,QSO,3.0,2,2.94,2.94,Noisy spectra bad classification and redshift...,4,none


In [82]:
#copy this text to a new cell to resolve the conflict
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 2
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'unconvincing high-z QSO, disagreement in flag. give flag 2.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)

,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
458,35185760101468965,-1,0.33.0.dev3890,0,QSO,2.94,PNT,2.0,RCS,0,...,22.810986,0.00,QSO,3.0,2,2.94,2.94,Noisy spectra bad classification and redshift...,4,"unconvincing high-z QSO, disagreement in flag...."
508,35185760101468965,-1,0.33.0.dev3890,0,QSO,2.94,VRK,2.0,--,--,...,22.810986,2.94,QSO,3.0,2,2.94,2.94,Noisy spectra bad classification and redshift...,4,"unconvincing high-z QSO, disagreement in flag...."
558,35185760101468965,-1,0.33.0.dev3890,0,QSO,2.94,ACE,2.0,--,--,...,22.810986,2.94,QSO,3.0,2,2.94,2.94,Noisy spectra bad classification and redshift...,4,"unconvincing high-z QSO, disagreement in flag...."
608,35185760101468965,-1,0.33.0.dev3890,0,QSO,2.94,TWL,2.0,--,--,...,22.810986,2.94,QSO,3.0,2,2.94,2.94,Noisy spectra bad classification and redshift...,4,"unconvincing high-z QSO, disagreement in flag...."


conflict ID =  27


In [83]:
#copy this text to a new cell to display the conflict
conflict_id=28
display_conflict(conflict_id, show_spectra=False)

,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
459,35185760101469754,GALAXY,1.579,PNT,1.0,S,Target,GALAXY,Noisy spectra possibly galaxy,0.000,GALAXY,1.75,3,1.579,1.579,Noisy spectra possibly galaxy| unconvincing d...,4,none
509,35185760101469754,GALAXY,1.579,VRK,0.0,--,--,--,unconvincing doublet (weak and in skyline noi...,1.579,GALAXY,1.75,3,1.579,1.579,Noisy spectra possibly galaxy| unconvincing d...,4,none
559,35185760101469754,GALAXY,1.579,ACE,3.0,--,--,--,--,1.579,GALAXY,1.75,3,1.579,1.579,Noisy spectra possibly galaxy| unconvincing d...,4,none
609,35185760101469754,GALAXY,1.579,TWL,3.0,--,--,--,--,1.579,GALAXY,1.75,3,1.579,1.579,Noisy spectra possibly galaxy| unconvincing d...,4,none


In [84]:
#copy this text to a new cell to resolve the conflict
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 1.579
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 1.75
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in flag. set to mean of 1.75'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
459,35185760101469754,-1,0.33.0.dev3890,0,GALAXY,1.579,PNT,1.75,S,Target,...,10.725617,1.579,GALAXY,1.75,3,1.579,1.579,Noisy spectra possibly galaxy| unconvincing d...,4,disagreement in flag. set to mean of 1.75
509,35185760101469754,-1,0.33.0.dev3890,0,GALAXY,1.579,VRK,1.75,--,--,...,10.725617,1.579,GALAXY,1.75,3,1.579,1.579,Noisy spectra possibly galaxy| unconvincing d...,4,disagreement in flag. set to mean of 1.75
559,35185760101469754,-1,0.33.0.dev3890,0,GALAXY,1.579,ACE,1.75,--,--,...,10.725617,1.579,GALAXY,1.75,3,1.579,1.579,Noisy spectra possibly galaxy| unconvincing d...,4,disagreement in flag. set to mean of 1.75
609,35185760101469754,-1,0.33.0.dev3890,0,GALAXY,1.579,TWL,1.75,--,--,...,10.725617,1.579,GALAXY,1.75,3,1.579,1.579,Noisy spectra possibly galaxy| unconvincing d...,4,disagreement in flag. set to mean of 1.75


conflict ID =  28


In [85]:
#copy this text to a new cell to display the conflict
conflict_id=29
display_conflict(conflict_id, show_spectra=False)


,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
460,35185760101469821,GALAXY,1.0058,PNT,4.0,RCS,--,--,Noisy spectra bad classification,1.0058,GALAXY,2.0,3,1.0058,0.0,Noisy spectra bad classification| unconvincing...,4,none
510,35185760101469821,GALAXY,1.0058,VRK,2.0,--,--,--,unconvincing doublet (two weak spikes above n...,1.0058,GALAXY,2.0,3,1.0058,0.0,Noisy spectra bad classification| unconvincing...,4,none
560,35185760101469821,GALAXY,1.0058,ACE,1.0,--,--,--,No clear match to features.,1.0058,GALAXY,2.0,3,1.0058,0.0,Noisy spectra bad classification| unconvincing...,4,none
610,35185760101469821,GALAXY,1.0058,TWL,1.0,--,--,--,very low S/N,1.0058,GALAXY,2.0,3,1.0058,0.0,Noisy spectra bad classification| unconvincing...,4,none


In [86]:
#copy this text to a new cell to resolve the conflict
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 2
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in flag. set to mean of 2.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)

,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
460,35185760101469821,-1,0.33.0.dev3890,0,GALAXY,1.0058,PNT,2.0,RCS,--,...,8.536425,1.0058,GALAXY,2.0,3,0.0,1.0058,Noisy spectra bad classification| unconvincing...,4,disagreement in flag. set to mean of 2.
510,35185760101469821,-1,0.33.0.dev3890,0,GALAXY,1.0058,VRK,2.0,--,--,...,8.536425,1.0058,GALAXY,2.0,3,0.0,1.0058,Noisy spectra bad classification| unconvincing...,4,disagreement in flag. set to mean of 2.
560,35185760101469821,-1,0.33.0.dev3890,0,GALAXY,1.0058,ACE,2.0,--,--,...,8.536425,1.0058,GALAXY,2.0,3,0.0,1.0058,Noisy spectra bad classification| unconvincing...,4,disagreement in flag. set to mean of 2.
610,35185760101469821,-1,0.33.0.dev3890,0,GALAXY,1.0058,TWL,2.0,--,--,...,8.536425,1.0058,GALAXY,2.0,3,0.0,1.0058,Noisy spectra bad classification| unconvincing...,4,disagreement in flag. set to mean of 2.


conflict ID =  29


In [87]:
#copy this text to a new cell to display the conflict
conflict_id=30
display_conflict(conflict_id, show_spectra=False)


,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
462,35185760101469927,GALAXY,0.9003,PNT,3.0,RCS,--,--,noisy spectra,0.9003,GALAXY,1.25,3,0.9003,0.0,noisy spectra| dubious fit (unsignificant dou...,4,none
512,35185760101469927,GALAXY,0.9003,VRK,0.0,RS,--,--,dubious fit (unsignificant doublet and strong...,0.9003,GALAXY,1.25,3,0.9003,0.0,noisy spectra| dubious fit (unsignificant dou...,4,none
562,35185760101469927,GALAXY,0.9003,ACE,1.0,--,--,--,No clear match to features.,0.9003,GALAXY,1.25,3,0.9003,0.0,noisy spectra| dubious fit (unsignificant dou...,4,none
612,35185760101469927,GALAXY,0.9003,TWL,1.0,--,--,--,One strong feature at 9520. Not sure which li...,0.9003,GALAXY,1.25,3,0.9003,0.0,noisy spectra| dubious fit (unsignificant dou...,4,none


In [88]:
#copy this text to a new cell to resolve the conflict
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 1.25
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'flag disagreement, set to mean flag 1.25'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)

,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
462,35185760101469927,-1,0.33.0.dev3890,0,GALAXY,0.9003,PNT,1.25,RCS,--,...,36.631807,0.9003,GALAXY,1.25,3,0.0,0.9003,noisy spectra| dubious fit (unsignificant dou...,4,"flag disagreement, set to mean flag 1.25"
512,35185760101469927,-1,0.33.0.dev3890,0,GALAXY,0.9003,VRK,1.25,RS,--,...,36.631807,0.9003,GALAXY,1.25,3,0.0,0.9003,noisy spectra| dubious fit (unsignificant dou...,4,"flag disagreement, set to mean flag 1.25"
562,35185760101469927,-1,0.33.0.dev3890,0,GALAXY,0.9003,ACE,1.25,--,--,...,36.631807,0.9003,GALAXY,1.25,3,0.0,0.9003,noisy spectra| dubious fit (unsignificant dou...,4,"flag disagreement, set to mean flag 1.25"
612,35185760101469927,-1,0.33.0.dev3890,0,GALAXY,0.9003,TWL,1.25,--,--,...,36.631807,0.9003,GALAXY,1.25,3,0.0,0.9003,noisy spectra| dubious fit (unsignificant dou...,4,"flag disagreement, set to mean flag 1.25"


conflict ID =  30


In [89]:
#copy this text to a new cell to display the conflict
conflict_id=31
display_conflict(conflict_id, show_spectra=False)


,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
467,35185760101470583,GALAXY,0.7884,PNT,0.0,--,--,--,noisy spectra,0.7884,GALAXY,1.0,3,0.7884,0.93318,noisy spectra| dubious fit (negative doublet ...,4,none
517,35185760101470583,GALAXY,0.7884,VRK,0.0,RS,--,--,dubious fit (negative doublet and OIII lines)...,0.7884,GALAXY,1.0,3,0.7884,0.93318,noisy spectra| dubious fit (negative doublet ...,4,none
567,35185760101470583,GALAXY,0.7884,ACE,3.0,--,2.4573,GALAXY,Based on bright Lyman alpha only.,2.4573,GALAXY,1.0,3,0.7884,0.93318,noisy spectra| dubious fit (negative doublet ...,4,none
617,35185760101470583,GALAXY,0.7884,TWL,1.0,--,--,--,Reverse model,0.7884,GALAXY,1.0,3,0.7884,0.93318,noisy spectra| dubious fit (negative doublet ...,4,none


In [90]:
#copy this text to a new cell to resolve the conflict
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 2.4573
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 2
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'one inspector suggests high-z based on LyA with high confidence. take with flag 2.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)



,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
467,35185760101470583,-1,0.33.0.dev3890,0,GALAXY,0.7884,PNT,2.0,--,--,...,10.138846,2.4573,GALAXY,1.0,3,0.93318,0.7884,noisy spectra| dubious fit (negative doublet ...,4,one inspector suggests high-z based on LyA wit...
517,35185760101470583,-1,0.33.0.dev3890,0,GALAXY,0.7884,VRK,2.0,RS,--,...,10.138846,2.4573,GALAXY,1.0,3,0.93318,0.7884,noisy spectra| dubious fit (negative doublet ...,4,one inspector suggests high-z based on LyA wit...
567,35185760101470583,-1,0.33.0.dev3890,0,GALAXY,0.7884,ACE,2.0,--,2.4573,...,10.138846,2.4573,GALAXY,1.0,3,0.93318,0.7884,noisy spectra| dubious fit (negative doublet ...,4,one inspector suggests high-z based on LyA wit...
617,35185760101470583,-1,0.33.0.dev3890,0,GALAXY,0.7884,TWL,2.0,--,--,...,10.138846,2.4573,GALAXY,1.0,3,0.93318,0.7884,noisy spectra| dubious fit (negative doublet ...,4,one inspector suggests high-z based on LyA wit...


conflict ID =  31


In [91]:
#copy this text to a new cell to display the conflict
conflict_id=32
display_conflict(conflict_id, show_spectra=False)

,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
471,35185760101471222,GALAXY,1.071,PNT,1.0,S,1.0710,GALAXY,noisy spectra,1.071,GALAXY,3.25,3,1.071,0.0,noisy spectra|unsecure doublet (weak and at t...,4,none
521,35185760101471222,GALAXY,1.071,VRK,4.0,--,--,--,unsecure doublet (weak and at the edge of red/...,1.071,GALAXY,3.25,3,1.071,0.0,noisy spectra|unsecure doublet (weak and at t...,4,none
571,35185760101471222,GALAXY,1.071,ACE,4.0,C,1.0710,QSO,Broad MgII but weak OII.,1.071,QSO,3.25,3,1.071,0.0,noisy spectra|unsecure doublet (weak and at t...,4,none
621,35185760101471222,GALAXY,1.071,TWL,4.0,--,--,--,Could be an AGM with broad MgII emission,1.071,GALAXY,3.25,3,1.071,0.0,noisy spectra|unsecure doublet (weak and at t...,4,none


In [92]:
#copy this text to a new cell to resolve the conflict
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 4
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 'QSO'

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in flag and spectype. z is secure. see broad MgII, set to QSO'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
471,35185760101471222,-1,0.33.0.dev3890,0,GALAXY,1.071,PNT,4.0,S,1.0710,...,250.577914,1.071,QSO,3.25,3,0.0,1.071,noisy spectra|unsecure doublet (weak and at t...,4,disagreement in flag and spectype. z is secure...
521,35185760101471222,-1,0.33.0.dev3890,0,GALAXY,1.071,VRK,4.0,--,--,...,250.577914,1.071,QSO,3.25,3,0.0,1.071,noisy spectra|unsecure doublet (weak and at t...,4,disagreement in flag and spectype. z is secure...
571,35185760101471222,-1,0.33.0.dev3890,0,GALAXY,1.071,ACE,4.0,C,1.0710,...,250.577914,1.071,QSO,3.25,3,0.0,1.071,noisy spectra|unsecure doublet (weak and at t...,4,disagreement in flag and spectype. z is secure...
621,35185760101471222,-1,0.33.0.dev3890,0,GALAXY,1.071,TWL,4.0,--,--,...,250.577914,1.071,QSO,3.25,3,0.0,1.071,noisy spectra|unsecure doublet (weak and at t...,4,disagreement in flag and spectype. z is secure...


conflict ID =  32


In [93]:
#copy this text to a new cell to display the conflict
conflict_id=33
display_conflict(conflict_id, show_spectra=False)


,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
472,35185760101471315,QSO,1.9673,PNT,1.0,S,1.9673,QSO,Very Noisy data,1.9673,QSO,3.25,3,1.9673,0.0,Very Noisy data| clear QSO|--|--,4,none
522,35185760101471315,QSO,1.9673,VRK,4.0,--,--,--,clear QSO,1.9673,QSO,3.25,3,1.9673,0.0,Very Noisy data| clear QSO|--|--,4,none
572,35185760101471315,QSO,1.9673,ACE,4.0,--,--,--,--,1.9673,QSO,3.25,3,1.9673,0.0,Very Noisy data| clear QSO|--|--,4,none
622,35185760101471315,QSO,1.9673,TWL,4.0,--,--,--,--,1.9673,QSO,3.25,3,1.9673,0.0,Very Noisy data| clear QSO|--|--,4,none


In [94]:
#copy this text to a new cell to resolve the conflict
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 4
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in flag but clear z.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)

,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
472,35185760101471315,-1,0.33.0.dev3890,0,QSO,1.9673,PNT,4.0,S,1.9673,...,297.568332,1.9673,QSO,3.25,3,0.0,1.9673,Very Noisy data| clear QSO|--|--,4,disagreement in flag but clear z.
522,35185760101471315,-1,0.33.0.dev3890,0,QSO,1.9673,VRK,4.0,--,--,...,297.568332,1.9673,QSO,3.25,3,0.0,1.9673,Very Noisy data| clear QSO|--|--,4,disagreement in flag but clear z.
572,35185760101471315,-1,0.33.0.dev3890,0,QSO,1.9673,ACE,4.0,--,--,...,297.568332,1.9673,QSO,3.25,3,0.0,1.9673,Very Noisy data| clear QSO|--|--,4,disagreement in flag but clear z.
622,35185760101471315,-1,0.33.0.dev3890,0,QSO,1.9673,TWL,4.0,--,--,...,297.568332,1.9673,QSO,3.25,3,0.0,1.9673,Very Noisy data| clear QSO|--|--,4,disagreement in flag but clear z.


conflict ID =  33


In [95]:
#copy this text to a new cell to display the conflict
conflict_id=34
display_conflict(conflict_id, show_spectra=False)


,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
477,35185760105661565,GALAXY,0.9664,PNT,1.0,S,0.9664,GALAXY,Noisy spectra,0.9664,GALAXY,3.0,3,0.9664,0.0,Noisy spectra|weak doublet but Balmer lines a...,4,none
527,35185760105661565,GALAXY,0.9664,VRK,3.0,--,--,--,weak doublet but Balmer lines are more convinc...,0.9664,GALAXY,3.0,3,0.9664,0.0,Noisy spectra|weak doublet but Balmer lines a...,4,none
577,35185760105661565,GALAXY,0.9664,ACE,4.0,--,--,--,--,0.9664,GALAXY,3.0,3,0.9664,0.0,Noisy spectra|weak doublet but Balmer lines a...,4,none
627,35185760105661565,GALAXY,0.9664,TWL,4.0,--,--,--,--,0.9664,GALAXY,3.0,3,0.9664,0.0,Noisy spectra|weak doublet but Balmer lines a...,4,none


In [96]:
#copy this text to a new cell to resolve the conflict
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 3
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in flag but z is secure.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)

,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
477,35185760105661565,-1,0.33.0.dev3890,0,GALAXY,0.9664,PNT,3.0,S,0.9664,...,124.07593,0.9664,GALAXY,3.0,3,0.0,0.9664,Noisy spectra|weak doublet but Balmer lines a...,4,disagreement in flag but z is secure.
527,35185760105661565,-1,0.33.0.dev3890,0,GALAXY,0.9664,VRK,3.0,--,--,...,124.07593,0.9664,GALAXY,3.0,3,0.0,0.9664,Noisy spectra|weak doublet but Balmer lines a...,4,disagreement in flag but z is secure.
577,35185760105661565,-1,0.33.0.dev3890,0,GALAXY,0.9664,ACE,3.0,--,--,...,124.07593,0.9664,GALAXY,3.0,3,0.0,0.9664,Noisy spectra|weak doublet but Balmer lines a...,4,disagreement in flag but z is secure.
627,35185760105661565,-1,0.33.0.dev3890,0,GALAXY,0.9664,TWL,3.0,--,--,...,124.07593,0.9664,GALAXY,3.0,3,0.0,0.9664,Noisy spectra|weak doublet but Balmer lines a...,4,disagreement in flag but z is secure.


conflict ID =  34


In [98]:
#copy this text to a new cell to display the conflict
conflict_id=35
display_conflict(conflict_id, show_spectra=False)


,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
481,35185760105662202,GALAXY,0.7661,PNT,1.0,--,0.7661,GALAXY,Very noisy data OIII line is resolved at λobs~...,0.7661,GALAXY,1.5,3,0.7661,0.342789,Very noisy data OIII line is resolved at λobs~...,4,none
531,35185760105662202,GALAXY,0.7661,VRK,2.0,R,1.3714,--,dubious fit (negative doublet) + VI redshift g...,1.3714,GALAXY,1.5,3,0.7661,0.342789,Very noisy data OIII line is resolved at λobs~...,4,none
581,35185760105662202,GALAXY,0.7661,ACE,3.0,R,1.3715,GALAXY,Redrock confuses OII for OIII,1.3715,GALAXY,1.5,3,0.7661,0.342789,Very noisy data OIII line is resolved at λobs~...,4,none
631,35185760105662202,GALAXY,0.7661,TWL,0.0,--,--,--,--,0.7661,GALAXY,1.5,3,0.7661,0.342789,Very noisy data OIII line is resolved at λobs~...,4,none


In [99]:
#copy this text to a new cell to resolve the conflict
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 1.3715
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 2
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'very low SNR. 2/4 inspectors suggest VI z, based on compromised OII on skylines. Take that with flag 2.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)

,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
481,35185760105662202,-1,0.33.0.dev3890,0,GALAXY,0.7661,PNT,2.0,--,0.7661,...,15.134168,1.3715,GALAXY,1.5,3,0.342789,0.7661,Very noisy data OIII line is resolved at λobs~...,4,"very low SNR. 2/4 inspectors suggest VI z, bas..."
531,35185760105662202,-1,0.33.0.dev3890,0,GALAXY,0.7661,VRK,2.0,R,1.3714,...,15.134168,1.3715,GALAXY,1.5,3,0.342789,0.7661,Very noisy data OIII line is resolved at λobs~...,4,"very low SNR. 2/4 inspectors suggest VI z, bas..."
581,35185760105662202,-1,0.33.0.dev3890,0,GALAXY,0.7661,ACE,2.0,R,1.3715,...,15.134168,1.3715,GALAXY,1.5,3,0.342789,0.7661,Very noisy data OIII line is resolved at λobs~...,4,"very low SNR. 2/4 inspectors suggest VI z, bas..."
631,35185760105662202,-1,0.33.0.dev3890,0,GALAXY,0.7661,TWL,2.0,--,--,...,15.134168,1.3715,GALAXY,1.5,3,0.342789,0.7661,Very noisy data OIII line is resolved at λobs~...,4,"very low SNR. 2/4 inspectors suggest VI z, bas..."


conflict ID =  35


In [100]:
#copy this text to a new cell to display the conflict
conflict_id=36
display_conflict(conflict_id, show_spectra=False)


,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
484,35185760105662698,GALAXY,0.8522,PNT,2.0,--,0.8522,GALAXY,Noisy spectra only OII line kinda resolved,0.8522,GALAXY,3.25,2,0.8522,0.0,Noisy spectra only OII line kinda resolved|th...,4,none
534,35185760105662698,GALAXY,0.8522,VRK,3.0,--,--,--,three-peak doublet but other (Balmer) lines lo...,0.8522,GALAXY,3.25,2,0.8522,0.0,Noisy spectra only OII line kinda resolved|th...,4,none
584,35185760105662698,GALAXY,0.8522,ACE,4.0,--,--,--,--,0.8522,GALAXY,3.25,2,0.8522,0.0,Noisy spectra only OII line kinda resolved|th...,4,none
634,35185760105662698,GALAXY,0.8522,TWL,4.0,--,--,--,--,0.8522,GALAXY,3.25,2,0.8522,0.0,Noisy spectra only OII line kinda resolved|th...,4,none


In [101]:
#copy this text to a new cell to resolve the conflict
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 3.25
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'flag disagreement. take mean.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)



,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
484,35185760105662698,-1,0.33.0.dev3890,0,GALAXY,0.8522,PNT,3.25,--,0.8522,...,119.946271,0.8522,GALAXY,3.25,2,0.0,0.8522,Noisy spectra only OII line kinda resolved|th...,4,flag disagreement. take mean.
534,35185760105662698,-1,0.33.0.dev3890,0,GALAXY,0.8522,VRK,3.25,--,--,...,119.946271,0.8522,GALAXY,3.25,2,0.0,0.8522,Noisy spectra only OII line kinda resolved|th...,4,flag disagreement. take mean.
584,35185760105662698,-1,0.33.0.dev3890,0,GALAXY,0.8522,ACE,3.25,--,--,...,119.946271,0.8522,GALAXY,3.25,2,0.0,0.8522,Noisy spectra only OII line kinda resolved|th...,4,flag disagreement. take mean.
634,35185760105662698,-1,0.33.0.dev3890,0,GALAXY,0.8522,TWL,3.25,--,--,...,119.946271,0.8522,GALAXY,3.25,2,0.0,0.8522,Noisy spectra only OII line kinda resolved|th...,4,flag disagreement. take mean.


conflict ID =  36


In [102]:
#copy this text to a new cell to display the conflict
conflict_id=37
display_conflict(conflict_id, show_spectra=False)


,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
488,35185760105663384,GALAXY,1.2636,PNT,1.0,S,1.2636,GALAXY,Noisy spectra,1.2636,GALAXY,3.0,3,1.2636,0.0,Noisy spectra| broad doublet but other (balme...,4,none
538,35185760105663384,GALAXY,1.2636,VRK,3.0,--,--,--,broad doublet but other (balmer) lines look ok,1.2636,GALAXY,3.0,3,1.2636,0.0,Noisy spectra| broad doublet but other (balme...,4,none
588,35185760105663384,GALAXY,1.2636,ACE,4.0,--,--,--,--,1.2636,GALAXY,3.0,3,1.2636,0.0,Noisy spectra| broad doublet but other (balme...,4,none
638,35185760105663384,GALAXY,1.2636,TWL,4.0,--,--,--,--,1.2636,GALAXY,3.0,3,1.2636,0.0,Noisy spectra| broad doublet but other (balme...,4,none


In [103]:
#copy this text to a new cell to resolve the conflict
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 3
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'flag disagreement. take mean.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)

,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
488,35185760105663384,-1,0.33.0.dev3890,0,GALAXY,1.2636,PNT,3.0,S,1.2636,...,365.477168,1.2636,GALAXY,3.0,3,0.0,1.2636,Noisy spectra| broad doublet but other (balme...,4,flag disagreement. take mean.
538,35185760105663384,-1,0.33.0.dev3890,0,GALAXY,1.2636,VRK,3.0,--,--,...,365.477168,1.2636,GALAXY,3.0,3,0.0,1.2636,Noisy spectra| broad doublet but other (balme...,4,flag disagreement. take mean.
588,35185760105663384,-1,0.33.0.dev3890,0,GALAXY,1.2636,ACE,3.0,--,--,...,365.477168,1.2636,GALAXY,3.0,3,0.0,1.2636,Noisy spectra| broad doublet but other (balme...,4,flag disagreement. take mean.
638,35185760105663384,-1,0.33.0.dev3890,0,GALAXY,1.2636,TWL,3.0,--,--,...,365.477168,1.2636,GALAXY,3.0,3,0.0,1.2636,Noisy spectra| broad doublet but other (balme...,4,flag disagreement. take mean.


conflict ID =  37


In [106]:
#copy this text to a new cell to display the conflict
conflict_id=38
display_conflict(conflict_id, show_spectra=False)

,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
489,35185760105663463,GALAXY,0.9563,PNT,0.0,RCS,--,--,Noisy spectra,0.9563,GALAXY,1.5,3,0.9563,0.190104,Noisy spectra| dubious fit (negative doublet ...,4,none
539,35185760105663463,GALAXY,0.9563,VRK,2.0,R,1.3279,--,dubious fit (negative doublet and OIII lines)...,1.3279,GALAXY,1.5,3,0.9563,0.190104,Noisy spectra| dubious fit (negative doublet ...,4,none
589,35185760105663463,GALAXY,0.9563,ACE,3.0,R,1.3282,GALAXY,Based on OII only.,1.3282,GALAXY,1.5,3,0.9563,0.190104,Noisy spectra| dubious fit (negative doublet ...,4,none
639,35185760105663463,GALAXY,0.9563,TWL,1.0,--,--,--,Low S/N,0.9563,GALAXY,1.5,3,0.9563,0.190104,Noisy spectra| dubious fit (negative doublet ...,4,none


In [107]:
#copy this text to a new cell to resolve the conflict
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 1.3280
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 2
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in flag and z. 2/4 inspectors suggest VI z based on dubious OII doublet. Take with flag 2.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
489,35185760105663463,-1,0.33.0.dev3890,0,GALAXY,0.9563,PNT,2.0,RCS,--,...,8.2686,1.328,GALAXY,1.5,3,0.190104,0.9563,Noisy spectra| dubious fit (negative doublet ...,4,disagreement in flag and z. 2/4 inspectors sug...
539,35185760105663463,-1,0.33.0.dev3890,0,GALAXY,0.9563,VRK,2.0,R,1.3279,...,8.2686,1.328,GALAXY,1.5,3,0.190104,0.9563,Noisy spectra| dubious fit (negative doublet ...,4,disagreement in flag and z. 2/4 inspectors sug...
589,35185760105663463,-1,0.33.0.dev3890,0,GALAXY,0.9563,ACE,2.0,R,1.3282,...,8.2686,1.328,GALAXY,1.5,3,0.190104,0.9563,Noisy spectra| dubious fit (negative doublet ...,4,disagreement in flag and z. 2/4 inspectors sug...
639,35185760105663463,-1,0.33.0.dev3890,0,GALAXY,0.9563,TWL,2.0,--,--,...,8.2686,1.328,GALAXY,1.5,3,0.190104,0.9563,Noisy spectra| dubious fit (negative doublet ...,4,disagreement in flag and z. 2/4 inspectors sug...


conflict ID =  38


In [110]:
#copy this text to a new cell to display the conflict
conflict_id=39
display_conflict(conflict_id, show_spectra=False)


,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
491,35185760105664072,GALAXY,1.46,PNT,2.0,--,1.2636,GALAXY,Noisy spectra OII is quite good resolved,1.2636,GALAXY,3.0,2,1.46,0.086764,Noisy spectra OII is quite good resolved|--|-...,4,none
541,35185760105664072,GALAXY,1.46,VRK,4.0,--,--,--,--,1.4600,GALAXY,3.0,2,1.46,0.086764,Noisy spectra OII is quite good resolved|--|-...,4,none
591,35185760105664072,GALAXY,1.46,ACE,3.0,--,--,--,--,1.4600,GALAXY,3.0,2,1.46,0.086764,Noisy spectra OII is quite good resolved|--|-...,4,none
641,35185760105664072,GALAXY,1.46,TWL,3.0,--,--,--,--,1.4600,GALAXY,3.0,2,1.46,0.086764,Noisy spectra OII is quite good resolved|--|-...,4,none


In [111]:
#copy this text to a new cell to resolve the conflict
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 1.46
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 4
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in z and flag. 00 bug.redrock z is secure.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)

,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
491,35185760105664072,-1,0.33.0.dev3890,0,GALAXY,1.46,PNT,4.0,--,1.2636,...,228.84382,1.46,GALAXY,3.0,2,0.086764,1.46,Noisy spectra OII is quite good resolved|--|-...,4,disagreement in z and flag. 00 bug.redrock z i...
541,35185760105664072,-1,0.33.0.dev3890,0,GALAXY,1.46,VRK,4.0,--,--,...,228.84382,1.46,GALAXY,3.0,2,0.086764,1.46,Noisy spectra OII is quite good resolved|--|-...,4,disagreement in z and flag. 00 bug.redrock z i...
591,35185760105664072,-1,0.33.0.dev3890,0,GALAXY,1.46,ACE,4.0,--,--,...,228.84382,1.46,GALAXY,3.0,2,0.086764,1.46,Noisy spectra OII is quite good resolved|--|-...,4,disagreement in z and flag. 00 bug.redrock z i...
641,35185760105664072,-1,0.33.0.dev3890,0,GALAXY,1.46,TWL,4.0,--,--,...,228.84382,1.46,GALAXY,3.0,2,0.086764,1.46,Noisy spectra OII is quite good resolved|--|-...,4,disagreement in z and flag. 00 bug.redrock z i...


conflict ID =  39


In [112]:
#copy this text to a new cell to display the conflict
conflict_id=40
display_conflict(conflict_id, show_spectra=False)



,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
494,35185760105664248,GALAXY,1.0035,PNT,2.0,--,Target,GALAXY,Noisy spectra only OII is resolved,0.0000,GALAXY,3.5,2,1.0035,1.0035,Noisy spectra only OII is resolved|--|--|--,4,none
544,35185760105664248,GALAXY,1.0035,VRK,4.0,--,--,--,--,1.0035,GALAXY,3.5,2,1.0035,1.0035,Noisy spectra only OII is resolved|--|--|--,4,none
594,35185760105664248,GALAXY,1.0035,ACE,4.0,--,--,--,--,1.0035,GALAXY,3.5,2,1.0035,1.0035,Noisy spectra only OII is resolved|--|--|--,4,none
644,35185760105664248,GALAXY,1.0035,TWL,4.0,--,--,--,--,1.0035,GALAXY,3.5,2,1.0035,1.0035,Noisy spectra only OII is resolved|--|--|--,4,none


In [113]:
#copy this text to a new cell to resolve the conflict
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 1.0035
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 4
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in flag. z is secure.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
494,35185760105664248,-1,0.33.0.dev3890,0,GALAXY,1.0035,PNT,4.0,--,Target,...,194.997913,1.0035,GALAXY,3.5,2,1.0035,1.0035,Noisy spectra only OII is resolved|--|--|--,4,disagreement in flag. z is secure.
544,35185760105664248,-1,0.33.0.dev3890,0,GALAXY,1.0035,VRK,4.0,--,--,...,194.997913,1.0035,GALAXY,3.5,2,1.0035,1.0035,Noisy spectra only OII is resolved|--|--|--,4,disagreement in flag. z is secure.
594,35185760105664248,-1,0.33.0.dev3890,0,GALAXY,1.0035,ACE,4.0,--,--,...,194.997913,1.0035,GALAXY,3.5,2,1.0035,1.0035,Noisy spectra only OII is resolved|--|--|--,4,disagreement in flag. z is secure.
644,35185760105664248,-1,0.33.0.dev3890,0,GALAXY,1.0035,TWL,4.0,--,--,...,194.997913,1.0035,GALAXY,3.5,2,1.0035,1.0035,Noisy spectra only OII is resolved|--|--|--,4,disagreement in flag. z is secure.


conflict ID =  40


In [116]:
#copy this text to a new cell to display the conflict
conflict_id= 41
display_conflict(conflict_id, show_spectra=False)


,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
497,35185760105664964,GALAXY,1.2121,PNT,1.0,S,1.2121,GALAXY,Noisy spectra only OII is kinda resoled,1.2121,GALAXY,2.0,2,1.211975,0.000226,Noisy spectra only OII is kinda resoled| broa...,4,none
547,35185760105664964,GALAXY,1.2121,VRK,2.0,--,--,--,broad weak doublet and weak Hgamm line only +...,1.2121,GALAXY,2.0,2,1.211975,0.000226,Noisy spectra only OII is kinda resoled| broa...,4,none
597,35185760105664964,GALAXY,1.2121,ACE,3.0,--,1.2116,GALAXY,--,1.2116,GALAXY,2.0,2,1.211975,0.000226,Noisy spectra only OII is kinda resoled| broa...,4,none
647,35185760105664964,GALAXY,1.2121,TWL,2.0,--,--,--,Weird best fit [OII] profile; being conservative,1.2121,GALAXY,2.0,2,1.211975,0.000226,Noisy spectra only OII is kinda resoled| broa...,4,none


In [117]:
#copy this text to a new cell to resolve the conflict
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 1.2116
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 2.5
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'flag disagreement, set to 2.5'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
497,35185760105664964,-1,0.33.0.dev3890,0,GALAXY,1.2121,PNT,2.5,S,1.2121,...,30.498181,1.2116,GALAXY,2.0,2,0.000226,1.211975,Noisy spectra only OII is kinda resoled| broa...,4,"flag disagreement, set to 2.5"
547,35185760105664964,-1,0.33.0.dev3890,0,GALAXY,1.2121,VRK,2.5,--,--,...,30.498181,1.2116,GALAXY,2.0,2,0.000226,1.211975,Noisy spectra only OII is kinda resoled| broa...,4,"flag disagreement, set to 2.5"
597,35185760105664964,-1,0.33.0.dev3890,0,GALAXY,1.2121,ACE,2.5,--,1.2116,...,30.498181,1.2116,GALAXY,2.0,2,0.000226,1.211975,Noisy spectra only OII is kinda resoled| broa...,4,"flag disagreement, set to 2.5"
647,35185760105664964,-1,0.33.0.dev3890,0,GALAXY,1.2121,TWL,2.5,--,--,...,30.498181,1.2116,GALAXY,2.0,2,0.000226,1.211975,Noisy spectra only OII is kinda resoled| broa...,4,"flag disagreement, set to 2.5"


conflict ID =  41


In [120]:
#copy this text to a new cell to display the conflict
conflict_id=42
display_conflict(conflict_id, show_spectra=False)



,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
693,35185760105665118,GALAXY,1.1442,TWL,3.0,--,--,--,[OII] on top of skylines,1.1442,GALAXY,2.333333,2,1.1442,0.0,[OII] on top of skylines|--| Noisy data possib...,3,none
742,35185760105665118,GALAXY,1.1442,ACE,3.0,--,--,--,--,1.1442,GALAXY,2.333333,2,1.1442,0.0,[OII] on top of skylines|--| Noisy data possib...,3,none
792,35185760105665118,GALAXY,1.1442,PNT,1.0,--,1.1442,GALAXY,Noisy data possibly a galaxy,1.1442,GALAXY,2.333333,2,1.1442,0.0,[OII] on top of skylines|--| Noisy data possib...,3,none


In [121]:
#copy this text to a new cell to resolve the conflict
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 3
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'flag disagreement. clear emission line. not resolved but change in continuum consistent with OII'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)

,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
693,35185760105665118,-1,0.33.0.dev3890,0,GALAXY,1.1442,TWL,3.0,--,--,...,102.108765,1.1442,GALAXY,2.333333,2,0.0,1.1442,[OII] on top of skylines|--| Noisy data possib...,3,flag disagreement. clear emission line. not re...
742,35185760105665118,-1,0.33.0.dev3890,0,GALAXY,1.1442,ACE,3.0,--,--,...,102.108765,1.1442,GALAXY,2.333333,2,0.0,1.1442,[OII] on top of skylines|--| Noisy data possib...,3,flag disagreement. clear emission line. not re...
792,35185760105665118,-1,0.33.0.dev3890,0,GALAXY,1.1442,PNT,3.0,--,1.1442,...,102.108765,1.1442,GALAXY,2.333333,2,0.0,1.1442,[OII] on top of skylines|--| Noisy data possib...,3,flag disagreement. clear emission line. not re...


conflict ID =  42


In [124]:
#copy this text to a new cell to display the conflict
conflict_id=43
display_conflict(conflict_id, show_spectra=False)



,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
694,35185760105665215,GALAXY,1.059,TWL,4.0,--,--,--,--,1.059,GALAXY,3.0,2,1.059,0.0,--|--| Noisy data only OII is resolved,3,none
743,35185760105665215,GALAXY,1.059,ACE,3.0,--,--,--,--,1.059,GALAXY,3.0,2,1.059,0.0,--|--| Noisy data only OII is resolved,3,none
793,35185760105665215,GALAXY,1.059,PNT,2.0,S,1.0590,GALAXY,Noisy data only OII is resolved,1.059,GALAXY,3.0,2,1.059,0.0,--|--| Noisy data only OII is resolved,3,none


In [125]:
#copy this text to a new cell to resolve the conflict
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 3
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'flag disagreement. set to mean.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)



,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
694,35185760105665215,-1,0.33.0.dev3890,0,GALAXY,1.059,TWL,3.0,--,--,...,204.248311,1.059,GALAXY,3.0,2,0.0,1.059,--|--| Noisy data only OII is resolved,3,flag disagreement. set to mean.
743,35185760105665215,-1,0.33.0.dev3890,0,GALAXY,1.059,ACE,3.0,--,--,...,204.248311,1.059,GALAXY,3.0,2,0.0,1.059,--|--| Noisy data only OII is resolved,3,flag disagreement. set to mean.
793,35185760105665215,-1,0.33.0.dev3890,0,GALAXY,1.059,PNT,3.0,S,1.0590,...,204.248311,1.059,GALAXY,3.0,2,0.0,1.059,--|--| Noisy data only OII is resolved,3,flag disagreement. set to mean.


conflict ID =  43


In [129]:
#copy this text to a new cell to display the conflict
conflict_id=44
display_conflict(conflict_id, show_spectra=False)

,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
695,35185760105665533,GALAXY,0.0058,TWL,4.0,--,--,--,--,0.0058,GALAXY,3.0,3,0.0058,0.0,--| Strong narrow lines including HeI|Wierd St...,3,none
744,35185760105665533,GALAXY,0.0058,ACE,4.0,--,--,--,Strong narrow lines including HeI,0.0058,GALAXY,3.0,3,0.0058,0.0,--| Strong narrow lines including HeI|Wierd St...,3,none
794,35185760105665533,GALAXY,0.0058,PNT,1.0,CS,0.0058,STAR,Wierd Star with emission lines detected I do n...,0.0058,STAR,3.0,3,0.0058,0.0,--| Strong narrow lines including HeI|Wierd St...,3,none


In [127]:
print(vi.loc[794]['VI comment'])

Wierd Star with emission lines detected I do not find any example in the Google Presentation


In [130]:
#copy this text to a new cell to resolve the conflict
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 4
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'flag disagreement. z is secure.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)



,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
695,35185760105665533,-1,0.33.0.dev3890,0,GALAXY,0.0058,TWL,4.0,--,--,...,22954.251607,0.0058,GALAXY,3.0,3,0.0,0.0058,--| Strong narrow lines including HeI|Wierd St...,3,flag disagreement. z is secure.
744,35185760105665533,-1,0.33.0.dev3890,0,GALAXY,0.0058,ACE,4.0,--,--,...,22954.251607,0.0058,GALAXY,3.0,3,0.0,0.0058,--| Strong narrow lines including HeI|Wierd St...,3,flag disagreement. z is secure.
794,35185760105665533,-1,0.33.0.dev3890,0,GALAXY,0.0058,PNT,4.0,CS,0.0058,...,22954.251607,0.0058,STAR,3.0,3,0.0,0.0058,--| Strong narrow lines including HeI|Wierd St...,3,flag disagreement. z is secure.


conflict ID =  44


In [133]:
#copy this text to a new cell to display the conflict
conflict_id=45
display_conflict(conflict_id, show_spectra=False)


,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
651,35185766095130548,QSO,0.9215,TWL,4.0,CS,--,GALAXY,Bad calibration feature 5800 A,0.9215,GALAXY,3.0,3,0.921667,0.00026,Bad calibration feature 5800 A| Poor continuu...,3,none
701,35185766095130548,QSO,0.9215,ACE,4.0,S,0.9220,GALAXY,Poor continuum subtraction on the red band.,0.9220,GALAXY,3.0,3,0.921667,0.00026,Bad calibration feature 5800 A| Poor continuu...,3,none
750,35185766095130548,QSO,0.9215,PNT,1.0,RCS,--,--,Bd spectrum,0.9215,QSO,3.0,3,0.921667,0.00026,Bad calibration feature 5800 A| Poor continuu...,3,none


In [135]:
#copy this text to a new cell to resolve the conflict
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 0.9220
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 3
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 'GALAXY'

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'bad extraction. OII and OIII, z is secure. change spectype to GALAXY'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
651,35185766095130548,-1,0.33.0.dev3890,0,QSO,0.9215,TWL,3.0,CS,--,...,107.823968,0.922,GALAXY,3.0,3,0.00026,0.921667,Bad calibration feature 5800 A| Poor continuu...,3,"bad extraction. OII and OIII, z is secure. cha..."
701,35185766095130548,-1,0.33.0.dev3890,0,QSO,0.9215,ACE,3.0,S,0.9220,...,107.823968,0.922,GALAXY,3.0,3,0.00026,0.921667,Bad calibration feature 5800 A| Poor continuu...,3,"bad extraction. OII and OIII, z is secure. cha..."
750,35185766095130548,-1,0.33.0.dev3890,0,QSO,0.9215,PNT,3.0,RCS,--,...,107.823968,0.922,GALAXY,3.0,3,0.00026,0.921667,Bad calibration feature 5800 A| Poor continuu...,3,"bad extraction. OII and OIII, z is secure. cha..."


conflict ID =  45


In [138]:
#copy this text to a new cell to display the conflict
conflict_id=46
display_conflict(conflict_id, show_spectra=False)



,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
652,35185766099323497,GALAXY,0.4462,TWL,2.0,S,0.9428,GALAXY,Bad calibration feature 5800 A,0.9428,GALAXY,2.0,2,0.4462,0.704052,Bad calibration feature 5800 A| Poor continuu...,3,none
702,35185766099323497,GALAXY,0.4462,ACE,3.0,RS,1.4644,GALAXY,Poor continuum subtraction on the red band.,1.4644,GALAXY,2.0,2,0.4462,0.704052,Bad calibration feature 5800 A| Poor continuu...,3,none
751,35185766099323497,GALAXY,0.4462,PNT,1.0,RCS,--,--,bad spectrum,0.4462,GALAXY,2.0,2,0.4462,0.704052,Bad calibration feature 5800 A| Poor continuu...,3,none


In [139]:
#copy this text to a new cell to resolve the conflict
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 1.4644
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 3
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in flag and z. 2 VI redshifts based on OII, but one is just skylines. take the other with flag 3.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
652,35185766099323497,-1,0.33.0.dev3890,0,GALAXY,0.4462,TWL,3.0,S,0.9428,...,124.66718,1.4644,GALAXY,2.0,2,0.704052,0.4462,Bad calibration feature 5800 A| Poor continuu...,3,disagreement in flag and z. 2 VI redshifts bas...
702,35185766099323497,-1,0.33.0.dev3890,0,GALAXY,0.4462,ACE,3.0,RS,1.4644,...,124.66718,1.4644,GALAXY,2.0,2,0.704052,0.4462,Bad calibration feature 5800 A| Poor continuu...,3,disagreement in flag and z. 2 VI redshifts bas...
751,35185766099323497,-1,0.33.0.dev3890,0,GALAXY,0.4462,PNT,3.0,RCS,--,...,124.66718,1.4644,GALAXY,2.0,2,0.704052,0.4462,Bad calibration feature 5800 A| Poor continuu...,3,disagreement in flag and z. 2 VI redshifts bas...


conflict ID =  46


In [142]:
#copy this text to a new cell to display the conflict
conflict_id=47
display_conflict(conflict_id, show_spectra=False)


,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
655,35185766099324463,GALAXY,0.9063,TWL,4.0,S,--,--,Bad calibration feature,0.9063,GALAXY,2.666667,4,0.9063,0.0,Bad calibration feature| Poor continuum subtr...,3,none
705,35185766099324463,GALAXY,0.9063,ACE,4.0,S,0.9063,GALAXY,Poor continuum subtraction on the red band.,0.9063,GALAXY,2.666667,4,0.9063,0.0,Bad calibration feature| Poor continuum subtr...,3,none
754,35185766099324463,GALAXY,0.9063,PNT,0.0,--,--,--,Bad calibration feature ~ Bad Spectrum,0.9063,GALAXY,2.666667,4,0.9063,0.0,Bad calibration feature| Poor continuum subtr...,3,none


In [143]:
#copy this text to a new cell to resolve the conflict
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 2
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'flag disagreement. issues in subtraction. one strong line but not resolved, not secure.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
655,35185766099324463,-1,0.33.0.dev3890,0,GALAXY,0.9063,TWL,2.0,S,--,...,393.501891,0.9063,GALAXY,2.666667,4,0.0,0.9063,Bad calibration feature| Poor continuum subtr...,3,flag disagreement. issues in subtraction. one ...
705,35185766099324463,-1,0.33.0.dev3890,0,GALAXY,0.9063,ACE,2.0,S,0.9063,...,393.501891,0.9063,GALAXY,2.666667,4,0.0,0.9063,Bad calibration feature| Poor continuum subtr...,3,flag disagreement. issues in subtraction. one ...
754,35185766099324463,-1,0.33.0.dev3890,0,GALAXY,0.9063,PNT,2.0,--,--,...,393.501891,0.9063,GALAXY,2.666667,4,0.0,0.9063,Bad calibration feature| Poor continuum subtr...,3,flag disagreement. issues in subtraction. one ...


conflict ID =  47


In [146]:
#copy this text to a new cell to display the conflict
conflict_id=48
display_conflict(conflict_id, show_spectra=False)



,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
656,35185766099324852,QSO,3.8212,TWL,1.0,CS,--,--,Bad calibration feature 5800 and 7600,3.8212,QSO,1.666667,4,3.8212,1.666888,Bad calibration feature 5800 and 7600| Poor ...,3,none
706,35185766099324852,QSO,3.8212,ACE,4.0,RCS,0.8078,GALAXY,Poor continuum subtraction on the red band.,0.8078,GALAXY,1.666667,4,3.8212,1.666888,Bad calibration feature 5800 and 7600| Poor ...,3,none
755,35185766099324852,QSO,3.8212,PNT,0.0,--,--,--,Bad spectrum,3.8212,QSO,1.666667,4,3.8212,1.666888,Bad calibration feature 5800 and 7600| Poor ...,3,none


In [147]:
#copy this text to a new cell to resolve the conflict
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 0.8078
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 3
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 'GALAXY'

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in z, flag and spectype. one inspector suggests VI z based on OII, weak balmer lines. take with flag 3. not a QSO. bad extraction.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)



,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
656,35185766099324852,-1,0.33.0.dev3890,0,QSO,3.8212,TWL,3.0,CS,--,...,2072.54065,0.8078,GALAXY,1.666667,4,1.666888,3.8212,Bad calibration feature 5800 and 7600| Poor ...,3,"disagreement in z, flag and spectype. one insp..."
706,35185766099324852,-1,0.33.0.dev3890,0,QSO,3.8212,ACE,3.0,RCS,0.8078,...,2072.54065,0.8078,GALAXY,1.666667,4,1.666888,3.8212,Bad calibration feature 5800 and 7600| Poor ...,3,"disagreement in z, flag and spectype. one insp..."
755,35185766099324852,-1,0.33.0.dev3890,0,QSO,3.8212,PNT,3.0,--,--,...,2072.54065,0.8078,GALAXY,1.666667,4,1.666888,3.8212,Bad calibration feature 5800 and 7600| Poor ...,3,"disagreement in z, flag and spectype. one insp..."


conflict ID =  48


In [152]:
#copy this text to a new cell to display the conflict
conflict_id=49
display_conflict(conflict_id, show_spectra=False)


,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
657,35185766099324872,QSO,3.7866,TWL,4.0,S,0.9020,GALAXY,Bad calibration feature 5800 A,0.9020,GALAXY,3.0,3,3.7866,1.516746,Bad calibration feature 5800 A| Poor continuu...,3,none
707,35185766099324872,QSO,3.7866,ACE,4.0,RCS,0.9019,GALAXY,Poor continuum subtraction on the red band.,0.9019,GALAXY,3.0,3,3.7866,1.516746,Bad calibration feature 5800 A| Poor continuu...,3,none
756,35185766099324872,QSO,3.7866,PNT,1.0,--,--,--,"Wierd bad spectrum"","" might worth investigatin...",3.7866,QSO,3.0,3,3.7866,1.516746,Bad calibration feature 5800 A| Poor continuu...,3,none


In [153]:
#copy this text to a new cell to resolve the conflict
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 0.9019
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 3
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 'GALAXY'

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in flag, z and spectype. bad extraction. 2/3 inspectors offer VI z based on OII and OIII, secure. not a QSO.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)



,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
657,35185766099324872,-1,0.33.0.dev3890,0,QSO,3.7866,TWL,3.0,S,0.9020,...,5834.085469,0.9019,GALAXY,3.0,3,1.516746,3.7866,Bad calibration feature 5800 A| Poor continuu...,3,"disagreement in flag, z and spectype. bad extr..."
707,35185766099324872,-1,0.33.0.dev3890,0,QSO,3.7866,ACE,3.0,RCS,0.9019,...,5834.085469,0.9019,GALAXY,3.0,3,1.516746,3.7866,Bad calibration feature 5800 A| Poor continuu...,3,"disagreement in flag, z and spectype. bad extr..."
756,35185766099324872,-1,0.33.0.dev3890,0,QSO,3.7866,PNT,3.0,--,--,...,5834.085469,0.9019,GALAXY,3.0,3,1.516746,3.7866,Bad calibration feature 5800 A| Poor continuu...,3,"disagreement in flag, z and spectype. bad extr..."


conflict ID =  49


In [154]:
#copy this text to a new cell to display the conflict
conflict_id= 50
display_conflict(conflict_id, show_spectra=False)



,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
660,35185766099325112,GALAXY,1.0281,TWL,4.0,--,--,--,Bad calibration feature 5800 A,1.0281,GALAXY,3.333333,2,1.0281,0.0,Bad calibration feature 5800 A| Poor continuu...,3,none
710,35185766099325112,GALAXY,1.0281,ACE,4.0,S,--,--,Poor continuum subtraction on the red band.,1.0281,GALAXY,3.333333,2,1.0281,0.0,Bad calibration feature 5800 A| Poor continuu...,3,none
759,35185766099325112,GALAXY,1.0281,PNT,2.0,S,1.0281,GALAXY,Noisy spectra but OII line is resolved,1.0281,GALAXY,3.333333,2,1.0281,0.0,Bad calibration feature 5800 A| Poor continuu...,3,none


In [155]:
#copy this text to a new cell to resolve the conflict
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 3.333333
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in flag. take the mean.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
660,35185766099325112,-1,0.33.0.dev3890,0,GALAXY,1.0281,TWL,3.333333,--,--,...,638.140576,1.0281,GALAXY,3.333333,2,0.0,1.0281,Bad calibration feature 5800 A| Poor continuu...,3,disagreement in flag. take the mean.
710,35185766099325112,-1,0.33.0.dev3890,0,GALAXY,1.0281,ACE,3.333333,S,--,...,638.140576,1.0281,GALAXY,3.333333,2,0.0,1.0281,Bad calibration feature 5800 A| Poor continuu...,3,disagreement in flag. take the mean.
759,35185766099325112,-1,0.33.0.dev3890,0,GALAXY,1.0281,PNT,3.333333,S,1.0281,...,638.140576,1.0281,GALAXY,3.333333,2,0.0,1.0281,Bad calibration feature 5800 A| Poor continuu...,3,disagreement in flag. take the mean.


conflict ID =  50


In [156]:
#copy this text to a new cell to display the conflict
conflict_id=51
display_conflict(conflict_id, show_spectra=False)

,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
661,35185766099325192,GALAXY,1.0099,TWL,4.0,--,--,--,Bad calibration feature 5800 A,1.0099,GALAXY,3.333333,2,1.0099,0.0,Bad calibration feature 5800 A| Poor continuu...,3,none
711,35185766099325192,GALAXY,1.0099,ACE,4.0,S,--,--,Poor continuum subtraction on the red band.,1.0099,GALAXY,3.333333,2,1.0099,0.0,Bad calibration feature 5800 A| Poor continuu...,3,none
760,35185766099325192,GALAXY,1.0099,PNT,2.0,--,1.0099,GALAXY,Noisy spectra but OII line is resolved,1.0099,GALAXY,3.333333,2,1.0099,0.0,Bad calibration feature 5800 A| Poor continuu...,3,none


In [157]:
#copy this text to a new cell to resolve the conflict
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 3.333333
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in flag. take the mean.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)

,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
661,35185766099325192,-1,0.33.0.dev3890,0,GALAXY,1.0099,TWL,3.333333,--,--,...,10.501782,1.0099,GALAXY,3.333333,2,0.0,1.0099,Bad calibration feature 5800 A| Poor continuu...,3,disagreement in flag. take the mean.
711,35185766099325192,-1,0.33.0.dev3890,0,GALAXY,1.0099,ACE,3.333333,S,--,...,10.501782,1.0099,GALAXY,3.333333,2,0.0,1.0099,Bad calibration feature 5800 A| Poor continuu...,3,disagreement in flag. take the mean.
760,35185766099325192,-1,0.33.0.dev3890,0,GALAXY,1.0099,PNT,3.333333,--,1.0099,...,10.501782,1.0099,GALAXY,3.333333,2,0.0,1.0099,Bad calibration feature 5800 A| Poor continuu...,3,disagreement in flag. take the mean.


conflict ID =  51


In [160]:
#copy this text to a new cell to display the conflict
conflict_id=52
display_conflict(conflict_id, show_spectra=False)


,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
662,35185766099325339,QSO,3.3147,TWL,4.0,--,--,--,"Model is bad. But"","" redshift seems to be right.",3.3147,QSO,3.0,3,3.3147,0.0,"Model is bad. But"","" redshift seems to be rig...",3,none
712,35185766099325339,QSO,3.3147,ACE,4.0,--,--,--,--,3.3147,QSO,3.0,3,3.3147,0.0,"Model is bad. But"","" redshift seems to be rig...",3,none
761,35185766099325339,QSO,3.3147,PNT,1.0,--,3.3147,QSO,Noisy spectra Possibly a QSO,3.3147,QSO,3.0,3,3.3147,0.0,"Model is bad. But"","" redshift seems to be rig...",3,none


In [161]:
#copy this text to a new cell to resolve the conflict
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 3
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in flag, set to mean.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)



,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
662,35185766099325339,-1,0.33.0.dev3890,0,QSO,3.3147,TWL,3.0,--,--,...,216.403045,3.3147,QSO,3.0,3,0.0,3.3147,"Model is bad. But"","" redshift seems to be rig...",3,"disagreement in flag, set to mean."
712,35185766099325339,-1,0.33.0.dev3890,0,QSO,3.3147,ACE,3.0,--,--,...,216.403045,3.3147,QSO,3.0,3,0.0,3.3147,"Model is bad. But"","" redshift seems to be rig...",3,"disagreement in flag, set to mean."
761,35185766099325339,-1,0.33.0.dev3890,0,QSO,3.3147,PNT,3.0,--,3.3147,...,216.403045,3.3147,QSO,3.0,3,0.0,3.3147,"Model is bad. But"","" redshift seems to be rig...",3,"disagreement in flag, set to mean."


conflict ID =  52


In [169]:
#copy this text to a new cell to display the conflict
conflict_id=53
display_conflict(conflict_id, show_spectra=False)

,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
664,35185766099325634,GALAXY,0.4503,TWL,2.5,--,1.4687,GALAXY,Possible [OII] emission at 9200A,1.4688,GALAXY,2.666667,1,0.4503,0.702268,Possible [OII] emission at 9200A| Poor contin...,3,disagreement in z and flag. 2/3 inspectors sug...
714,35185766099325634,GALAXY,0.4503,ACE,2.5,--,1.4688,GALAXY,Poor continuum subtraction on the red band.,1.4688,GALAXY,2.666667,1,0.4503,0.702268,Possible [OII] emission at 9200A| Poor contin...,3,disagreement in z and flag. 2/3 inspectors sug...
763,35185766099325634,GALAXY,0.4503,PNT,2.5,RCS,--,--,Noisy spectra bad calibration feature at λobs...,1.4688,GALAXY,2.666667,1,0.4503,0.702268,Possible [OII] emission at 9200A| Poor contin...,3,disagreement in z and flag. 2/3 inspectors sug...


In [172]:
#copy this text to a new cell to resolve the conflict
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 1.4688
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 2.5
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in z and flag. 2/3 inspectors suggets VI z based on OII. take it with their mean flag 2.5.'


#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
664,35185766099325634,-1,0.33.0.dev3890,0,GALAXY,0.4503,TWL,2.5,--,1.4687,...,22.401379,1.4688,GALAXY,2.666667,1,0.702268,0.4503,Possible [OII] emission at 9200A| Poor contin...,3,disagreement in z and flag. 2/3 inspectors sug...
714,35185766099325634,-1,0.33.0.dev3890,0,GALAXY,0.4503,ACE,2.5,--,1.4688,...,22.401379,1.4688,GALAXY,2.666667,1,0.702268,0.4503,Possible [OII] emission at 9200A| Poor contin...,3,disagreement in z and flag. 2/3 inspectors sug...
763,35185766099325634,-1,0.33.0.dev3890,0,GALAXY,0.4503,PNT,2.5,RCS,--,...,22.401379,1.4688,GALAXY,2.666667,1,0.702268,0.4503,Possible [OII] emission at 9200A| Poor contin...,3,disagreement in z and flag. 2/3 inspectors sug...


conflict ID =  53


In [175]:
#copy this text to a new cell to display the conflict
conflict_id=54
display_conflict(conflict_id, show_spectra=False)


,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
670,35185766103516163,GALAXY,0.4598,TWL,2.0,--,--,--,[OIII] emission line next to skylines / or [O...,0.4598,GALAXY,2.666667,2,0.4598,0.343266,[OIII] emission line next to skylines / or [O...,3,none
720,35185766103516163,GALAXY,0.4598,ACE,4.0,--,0.9609,--,Redrock confuses OII for OIII,0.9609,GALAXY,2.666667,2,0.4598,0.343266,[OIII] emission line next to skylines / or [O...,3,none
769,35185766103516163,GALAXY,0.4598,PNT,2.0,--,0.4598,GALAXY,Bad spectra with one of the doublets OIII bein...,0.4598,GALAXY,2.666667,2,0.4598,0.343266,[OIII] emission line next to skylines / or [O...,3,none


In [176]:
#copy this text to a new cell to resolve the conflict
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 0.9609
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 2
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in flag and z. 1/3 inspector gives VI z based on OII but doublet unresolved, could be OIII as per rr answer. take VI with flag 2. bad extraction.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
670,35185766103516163,-1,0.33.0.dev3890,0,GALAXY,0.4598,TWL,2.0,--,--,...,45.46605,0.9609,GALAXY,2.666667,2,0.343266,0.4598,[OIII] emission line next to skylines / or [O...,3,disagreement in flag and z. 1/3 inspector give...
720,35185766103516163,-1,0.33.0.dev3890,0,GALAXY,0.4598,ACE,2.0,--,0.9609,...,45.46605,0.9609,GALAXY,2.666667,2,0.343266,0.4598,[OIII] emission line next to skylines / or [O...,3,disagreement in flag and z. 1/3 inspector give...
769,35185766103516163,-1,0.33.0.dev3890,0,GALAXY,0.4598,PNT,2.0,--,0.4598,...,45.46605,0.9609,GALAXY,2.666667,2,0.343266,0.4598,[OIII] emission line next to skylines / or [O...,3,disagreement in flag and z. 1/3 inspector give...


conflict ID =  54


In [179]:
#copy this text to a new cell to display the conflict
conflict_id=55
display_conflict(conflict_id, show_spectra=False)

,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
674,35185766103516951,GALAXY,0.6145,TWL,4.0,R,1.1686,GALAXY,Bad calibration,1.1686,GALAXY,2.666667,3,0.6145,0.343202,Bad calibration| Redrock confuses OII for OII...,3,none
724,35185766103516951,GALAXY,0.6145,ACE,3.0,R,1.1686,GALAXY,Redrock confuses OII for OIII,1.1686,GALAXY,2.666667,3,0.6145,0.343202,Bad calibration| Redrock confuses OII for OII...,3,none
773,35185766103516951,GALAXY,0.6145,PNT,1.0,S,0.6145,GALAXY,Noisy data possibly an ELG,0.6145,GALAXY,2.666667,3,0.6145,0.343202,Bad calibration| Redrock confuses OII for OII...,3,none


In [180]:
#copy this text to a new cell to resolve the conflict
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 1.1686
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 3
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'bad calibration. disagreement in z and flag. 2/3 inspectors identify VI z on doublet. secure.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)

,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
674,35185766103516951,-1,0.33.0.dev3890,0,GALAXY,0.6145,TWL,3.0,R,1.1686,...,128.928454,1.1686,GALAXY,2.666667,3,0.343202,0.6145,Bad calibration| Redrock confuses OII for OII...,3,bad calibration. disagreement in z and flag. 2...
724,35185766103516951,-1,0.33.0.dev3890,0,GALAXY,0.6145,ACE,3.0,R,1.1686,...,128.928454,1.1686,GALAXY,2.666667,3,0.343202,0.6145,Bad calibration| Redrock confuses OII for OII...,3,bad calibration. disagreement in z and flag. 2...
773,35185766103516951,-1,0.33.0.dev3890,0,GALAXY,0.6145,PNT,3.0,S,0.6145,...,128.928454,1.1686,GALAXY,2.666667,3,0.343202,0.6145,Bad calibration| Redrock confuses OII for OII...,3,bad calibration. disagreement in z and flag. 2...


conflict ID =  55


In [183]:
#copy this text to a new cell to display the conflict
conflict_id=56
display_conflict(conflict_id, show_spectra=False)

,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
675,35185766103517062,QSO,1.3353,TWL,1.0,--,--,--,Low S/N,1.3353,QSO,1.666667,2,1.3353,0.051937,Low S/N| And a white dwarf in the blue with z...,3,none
725,35185766103517062,QSO,1.3353,ACE,3.0,R,1.2200,GALAXY,And a white dwarf in the blue with z=0.0025,1.2200,GALAXY,1.666667,2,1.3353,0.051937,Low S/N| And a white dwarf in the blue with z...,3,none
774,35185766103517062,QSO,1.3353,PNT,1.0,S,1.3353,QSO,Noisy data,1.3353,QSO,1.666667,2,1.3353,0.051937,Low S/N| And a white dwarf in the blue with z...,3,none


In [184]:
#copy this text to a new cell to resolve the conflict
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 1.2200
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 3
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 'GALAXY'

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in spectype, z and flag. 1/3 inspectors suggests VI z based on unresolved weak doublet. take with flag 2. not a QSO. bad extraction.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)



,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
675,35185766103517062,-1,0.33.0.dev3890,0,QSO,1.3353,TWL,3.0,--,--,...,1.946532,1.22,GALAXY,1.666667,2,0.051937,1.3353,Low S/N| And a white dwarf in the blue with z...,3,"disagreement in spectype, z and flag. 1/3 insp..."
725,35185766103517062,-1,0.33.0.dev3890,0,QSO,1.3353,ACE,3.0,R,1.2200,...,1.946532,1.22,GALAXY,1.666667,2,0.051937,1.3353,Low S/N| And a white dwarf in the blue with z...,3,"disagreement in spectype, z and flag. 1/3 insp..."
774,35185766103517062,-1,0.33.0.dev3890,0,QSO,1.3353,PNT,3.0,S,1.3353,...,1.946532,1.22,GALAXY,1.666667,2,0.051937,1.3353,Low S/N| And a white dwarf in the blue with z...,3,"disagreement in spectype, z and flag. 1/3 insp..."


conflict ID =  56


In [187]:
#copy this text to a new cell to display the conflict
conflict_id=57
display_conflict(conflict_id, show_spectra=False)

,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
677,35185766103517283,GALAXY,0.7508,TWL,4.0,--,--,--,--,0.7508,GALAXY,3.333333,2,0.751033,0.0004,--| MgII in absorption| Noisy data,3,none
726,35185766103517283,GALAXY,0.7508,ACE,4.0,--,0.7515,GALAXY,MgII in absorption,0.7515,GALAXY,3.333333,2,0.751033,0.0004,--| MgII in absorption| Noisy data,3,none
776,35185766103517283,GALAXY,0.7508,PNT,2.0,S,0.7508,GALAXY,Noisy data,0.7508,GALAXY,3.333333,2,0.751033,0.0004,--| MgII in absorption| Noisy data,3,none


In [188]:
#copy this text to a new cell to resolve the conflict
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 3.333333
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'flag disagreement. take mean.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)

,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
677,35185766103517283,-1,0.33.0.dev3890,0,GALAXY,0.7508,TWL,3.333333,--,--,...,179.892299,0.7508,GALAXY,3.333333,2,0.0004,0.751033,--| MgII in absorption| Noisy data,3,flag disagreement. take mean.
726,35185766103517283,-1,0.33.0.dev3890,0,GALAXY,0.7508,ACE,3.333333,--,0.7515,...,179.892299,0.7515,GALAXY,3.333333,2,0.0004,0.751033,--| MgII in absorption| Noisy data,3,flag disagreement. take mean.
776,35185766103517283,-1,0.33.0.dev3890,0,GALAXY,0.7508,PNT,3.333333,S,0.7508,...,179.892299,0.7508,GALAXY,3.333333,2,0.0004,0.751033,--| MgII in absorption| Noisy data,3,flag disagreement. take mean.


conflict ID =  57


In [191]:
#copy this text to a new cell to display the conflict
conflict_id=58
display_conflict(conflict_id, show_spectra=False)

,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
678,35185766103517701,GALAXY,0.6193,TWL,4.0,R,1.1751,--,Bad continuum calibration,1.1751,GALAXY,3.666667,1,0.6193,0.343235,Bad continuum calibration|Redrock confuses OI...,3,none
727,35185766103517701,GALAXY,0.6193,ACE,4.0,R,1.1751,GALAXY,Redrock confuses OII for OIII,1.1751,GALAXY,3.666667,1,0.6193,0.343235,Bad continuum calibration|Redrock confuses OI...,3,none
777,35185766103517701,GALAXY,0.6193,PNT,3.0,--,0.6193,GALAXY,Noisy spectra with the OIII is resolved only a...,0.6193,GALAXY,3.666667,1,0.6193,0.343235,Bad continuum calibration|Redrock confuses OI...,3,none


In [192]:
#copy this text to a new cell to resolve the conflict
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 1.1751
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 4
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in redshift. 2/3 inspectors give VI based on secure doublet.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)

,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
678,35185766103517701,-1,0.33.0.dev3890,0,GALAXY,0.6193,TWL,4.0,R,1.1751,...,165.131892,1.1751,GALAXY,3.666667,1,0.343235,0.6193,Bad continuum calibration|Redrock confuses OI...,3,disagreement in redshift. 2/3 inspectors give ...
727,35185766103517701,-1,0.33.0.dev3890,0,GALAXY,0.6193,ACE,4.0,R,1.1751,...,165.131892,1.1751,GALAXY,3.666667,1,0.343235,0.6193,Bad continuum calibration|Redrock confuses OI...,3,disagreement in redshift. 2/3 inspectors give ...
777,35185766103517701,-1,0.33.0.dev3890,0,GALAXY,0.6193,PNT,4.0,--,0.6193,...,165.131892,1.1751,GALAXY,3.666667,1,0.343235,0.6193,Bad continuum calibration|Redrock confuses OI...,3,disagreement in redshift. 2/3 inspectors give ...


conflict ID =  58


In [195]:
#copy this text to a new cell to display the conflict
conflict_id=59
display_conflict(conflict_id, show_spectra=False)


,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
679,35185766103518381,GALAXY,0.4615,TWL,2.0,R,0.9632,--,Possible [OII] at z=0.9632 but with strange li...,0.9632,GALAXY,2.666667,2,0.4615,0.343277,Possible [OII] at z=0.9632 but with strange li...,3,none
728,35185766103518381,GALAXY,0.4615,ACE,4.0,R,0.9632,GALAXY,--,0.9632,GALAXY,2.666667,2,0.4615,0.343277,Possible [OII] at z=0.9632 but with strange li...,3,none
778,35185766103518381,GALAXY,0.4615,PNT,2.0,--,0.4615,GALAXY,Noisy spectra,0.4615,GALAXY,2.666667,2,0.4615,0.343277,Possible [OII] at z=0.9632 but with strange li...,3,none


In [196]:
print(vi.loc[679]['VI comment'])

Possible [OII] at z=0.9632 but with strange line shapes


In [197]:
#copy this text to a new cell to resolve the conflict
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 0.9632
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 3
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in z and flag. 2/3 inspectors suggest VI z based on doublet. take it with their mean flag.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)

,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
679,35185766103518381,-1,0.33.0.dev3890,0,GALAXY,0.4615,TWL,3.0,R,0.9632,...,128.885017,0.9632,GALAXY,2.666667,2,0.343277,0.4615,Possible [OII] at z=0.9632 but with strange li...,3,disagreement in z and flag. 2/3 inspectors sug...
728,35185766103518381,-1,0.33.0.dev3890,0,GALAXY,0.4615,ACE,3.0,R,0.9632,...,128.885017,0.9632,GALAXY,2.666667,2,0.343277,0.4615,Possible [OII] at z=0.9632 but with strange li...,3,disagreement in z and flag. 2/3 inspectors sug...
778,35185766103518381,-1,0.33.0.dev3890,0,GALAXY,0.4615,PNT,3.0,--,0.4615,...,128.885017,0.9632,GALAXY,2.666667,2,0.343277,0.4615,Possible [OII] at z=0.9632 but with strange li...,3,disagreement in z and flag. 2/3 inspectors sug...


conflict ID =  59


In [200]:
#copy this text to a new cell to display the conflict
conflict_id=60
display_conflict(conflict_id, show_spectra=False)


,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
681,35185766103519454,QSO,1.6468,TWL,0.0,RC,--,--,Bad calibration,1.6468,QSO,1.0,3,1.6468,0.08877,Bad calibration|--| Noisy spectra,3,none
730,35185766103519454,QSO,1.6468,ACE,3.0,R,1.4310,GALAXY,--,1.4310,GALAXY,1.0,3,1.6468,0.08877,Bad calibration|--| Noisy spectra,3,none
780,35185766103519454,QSO,1.6468,PNT,0.0,RC,--,--,Noisy spectra,1.6468,QSO,1.0,3,1.6468,0.08877,Bad calibration|--| Noisy spectra,3,none


In [202]:
#copy this text to a new cell to resolve the conflict
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 1.4310
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 2
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 'GALAXY'

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in spectype, flag and z. 1/3 inspectors gives VI z based on dubious OII. take with flag 2. not a QSO. bad reduction.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)

,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
681,35185766103519454,-1,0.33.0.dev3890,0,QSO,1.6468,TWL,2.0,RC,--,...,21.508393,1.431,GALAXY,1.0,3,0.08877,1.6468,Bad calibration|--| Noisy spectra,3,"disagreement in spectype, flag and z. 1/3 insp..."
730,35185766103519454,-1,0.33.0.dev3890,0,QSO,1.6468,ACE,2.0,R,1.4310,...,21.508393,1.431,GALAXY,1.0,3,0.08877,1.6468,Bad calibration|--| Noisy spectra,3,"disagreement in spectype, flag and z. 1/3 insp..."
780,35185766103519454,-1,0.33.0.dev3890,0,QSO,1.6468,PNT,2.0,RC,--,...,21.508393,1.431,GALAXY,1.0,3,0.08877,1.6468,Bad calibration|--| Noisy spectra,3,"disagreement in spectype, flag and z. 1/3 insp..."


conflict ID =  60


In [205]:
#copy this text to a new cell to display the conflict
conflict_id=61
display_conflict(conflict_id, show_spectra=False)

,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
684,35185766107710247,QSO,1.7265,TWL,0.0,--,--,--,Very low S/N,1.7265,QSO,1.0,3,1.7265,0.038153,Very low S/N| OII in sky lines| Noisy spectra,3,none
733,35185766107710247,QSO,1.7265,ACE,3.0,R,1.6263,GALAXY,OII in sky lines,1.6263,GALAXY,1.0,3,1.7265,0.038153,Very low S/N| OII in sky lines| Noisy spectra,3,none
783,35185766107710247,QSO,1.7265,PNT,0.0,RCS,1.7265,GALAXY,Noisy spectra,1.7265,GALAXY,1.0,3,1.7265,0.038153,Very low S/N| OII in sky lines| Noisy spectra,3,none


In [206]:
#copy this text to a new cell to resolve the conflict
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 1.6263
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 2
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 'GALAXY'

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in flag, spectype and z. 1/3 inspectors give VI z based on OII in skylines. take with flag 2. not a QSO. bad extraction.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
684,35185766107710247,-1,0.33.0.dev3890,0,QSO,1.7265,TWL,2.0,--,--,...,3.281282,1.6263,GALAXY,1.0,3,0.038153,1.7265,Very low S/N| OII in sky lines| Noisy spectra,3,"disagreement in flag, spectype and z. 1/3 insp..."
733,35185766107710247,-1,0.33.0.dev3890,0,QSO,1.7265,ACE,2.0,R,1.6263,...,3.281282,1.6263,GALAXY,1.0,3,0.038153,1.7265,Very low S/N| OII in sky lines| Noisy spectra,3,"disagreement in flag, spectype and z. 1/3 insp..."
783,35185766107710247,-1,0.33.0.dev3890,0,QSO,1.7265,PNT,2.0,RCS,1.7265,...,3.281282,1.6263,GALAXY,1.0,3,0.038153,1.7265,Very low S/N| OII in sky lines| Noisy spectra,3,"disagreement in flag, spectype and z. 1/3 insp..."


conflict ID =  61


In [211]:
#copy this text to a new cell to display the conflict
conflict_id=62
display_conflict(conflict_id, show_spectra=False)

,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
686,35185766107712826,GALAXY,0.6726,TWL,2.0,R,1.2467,--,Likely [OII] emission at 8380 A,1.2467,GALAXY,2.666667,2,0.6726,0.343238,Likely [OII] emission at 8380 A|And has secon...,3,none
735,35185766107712826,GALAXY,0.6726,ACE,4.0,R,0.7971,GALAXY,And has second redshift of z=1.2466 with stron...,0.7971,GALAXY,2.666667,2,0.6726,0.343238,Likely [OII] emission at 8380 A|And has secon...,3,none
785,35185766107712826,GALAXY,0.6726,PNT,2.0,--,0.6726,GALAXY,Noisy spectra One of the doublets OIII is reso...,0.6726,GALAXY,2.666667,2,0.6726,0.343238,Likely [OII] emission at 8380 A|And has secon...,3,none


In [212]:
#copy this text to a new cell to resolve the conflict
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] =0.7971
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 3
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'looks like two redshifts, both in ELG range. take one at 0.7971, showing OII and OIII'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
686,35185766107712826,-1,0.33.0.dev3890,0,GALAXY,0.6726,TWL,3.0,R,1.2467,...,161.929006,0.7971,GALAXY,2.666667,2,0.343238,0.6726,Likely [OII] emission at 8380 A|And has secon...,3,"looks like two redshifts, both in ELG range. t..."
735,35185766107712826,-1,0.33.0.dev3890,0,GALAXY,0.6726,ACE,3.0,R,0.7971,...,161.929006,0.7971,GALAXY,2.666667,2,0.343238,0.6726,Likely [OII] emission at 8380 A|And has secon...,3,"looks like two redshifts, both in ELG range. t..."
785,35185766107712826,-1,0.33.0.dev3890,0,GALAXY,0.6726,PNT,3.0,--,0.6726,...,161.929006,0.7971,GALAXY,2.666667,2,0.343238,0.6726,Likely [OII] emission at 8380 A|And has secon...,3,"looks like two redshifts, both in ELG range. t..."


conflict ID =  62


In [215]:
#copy this text to a new cell to display the conflict
conflict_id=63
display_conflict(conflict_id, show_spectra=False)


,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
688,35185766107713825,GALAXY,0.6724,TWL,3.0,R,1.2465,--,--,1.2465,GALAXY,1.666667,2,0.6724,0.343279,--| No clear match to features.| Noisy data on...,3,none
737,35185766107713825,GALAXY,0.6724,ACE,1.0,--,--,--,No clear match to features.,0.6724,GALAXY,1.666667,2,0.6724,0.343279,--| No clear match to features.| Noisy data on...,3,none
787,35185766107713825,GALAXY,0.6724,PNT,1.0,S,0.6724,GALAXY,Noisy data only on of the doublets OIII is re...,0.6724,GALAXY,1.666667,2,0.6724,0.343279,--| No clear match to features.| Noisy data on...,3,none


In [216]:
#copy this text to a new cell to resolve the conflict
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 1.2465
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 3
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in z and flag. 1/3 inspectors suggests VI z based on doublet. take with flag 3.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
688,35185766107713825,-1,0.33.0.dev3890,0,GALAXY,0.6724,TWL,3.0,R,1.2465,...,38.776628,1.2465,GALAXY,1.666667,2,0.343279,0.6724,--| No clear match to features.| Noisy data on...,3,disagreement in z and flag. 1/3 inspectors sug...
737,35185766107713825,-1,0.33.0.dev3890,0,GALAXY,0.6724,ACE,3.0,--,--,...,38.776628,1.2465,GALAXY,1.666667,2,0.343279,0.6724,--| No clear match to features.| Noisy data on...,3,disagreement in z and flag. 1/3 inspectors sug...
787,35185766107713825,-1,0.33.0.dev3890,0,GALAXY,0.6724,PNT,3.0,S,0.6724,...,38.776628,1.2465,GALAXY,1.666667,2,0.343279,0.6724,--| No clear match to features.| Noisy data on...,3,disagreement in z and flag. 1/3 inspectors sug...


conflict ID =  63


In [219]:
#copy this text to a new cell to display the conflict
conflict_id=64
display_conflict(conflict_id, show_spectra=False)

,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
689,35185766107714202,QSO,1.6755,TWL,2.0,RC,1.5348,GALAXY,Likely [OII] emission at 9450 but in skyline ...,1.5348,GALAXY,2.0,4,1.6755,0.055632,Likely [OII] emission at 9450 but in skyline ...,3,none
738,35185766107714202,QSO,1.6755,ACE,4.0,RC,1.5345,GALAXY,MgII in absorption,1.5345,GALAXY,2.0,4,1.6755,0.055632,Likely [OII] emission at 9450 but in skyline ...,3,none
788,35185766107714202,QSO,1.6755,PNT,0.0,RCS,--,--,Noisy data,1.6755,QSO,2.0,4,1.6755,0.055632,Likely [OII] emission at 9450 but in skyline ...,3,none


In [220]:
#copy this text to a new cell to resolve the conflict
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 1.5346
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 3
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 'GALAXY'

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in z, flag and spectype. 2/3 inspectors give VI based on doublet in skylines. take with their mean flag. not a QSO. bad extraction.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)

,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
689,35185766107714202,-1,0.33.0.dev3890,0,QSO,1.6755,TWL,3.0,RC,1.5348,...,24.962845,1.5346,GALAXY,2.0,4,0.055632,1.6755,Likely [OII] emission at 9450 but in skyline ...,3,"disagreement in z, flag and spectype. 2/3 insp..."
738,35185766107714202,-1,0.33.0.dev3890,0,QSO,1.6755,ACE,3.0,RC,1.5345,...,24.962845,1.5346,GALAXY,2.0,4,0.055632,1.6755,Likely [OII] emission at 9450 but in skyline ...,3,"disagreement in z, flag and spectype. 2/3 insp..."
788,35185766107714202,-1,0.33.0.dev3890,0,QSO,1.6755,PNT,3.0,RCS,--,...,24.962845,1.5346,GALAXY,2.0,4,0.055632,1.6755,Likely [OII] emission at 9450 but in skyline ...,3,"disagreement in z, flag and spectype. 2/3 insp..."


conflict ID =  64


### and so on...

We should now recompute the conflicts, and not find any.

In [224]:
vi_conflict = find_conflicts(vi_gp)

In [225]:
unique_targets = np.unique(vi_conflict['TargetID'].tolist())
print('Targets with problematic VI: ', unique_targets)
print('Total number of conflicts to resolve: ', len(unique_targets))

Targets with problematic VI:  [35185760076305772 35185760105665533]
Total number of conflicts to resolve:  2


Display anything that was missed (if "Total number of conflicts" isn't zero) and resolve!

In [226]:
#copy this text to a new cell to display the conflict
conflict_id=0
display_conflict(conflict_id)

,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
45,35185760076305772,GALAXY,0.009,DRP,3.5,--,--,--,--,0.0090,GALAXY,3.0,2,0.009,0.687611,--|dubious fit + VI redshift gives resolved do...,3,disagreement in z and flag. 2/3 of inspectors ...
95,35185760076305772,GALAXY,0.009,vrk,3.5,R,0.7028,--,dubious fit + VI redshift gives resolved doubl...,0.7028,GALAXY,3.0,2,0.009,0.687611,--|dubious fit + VI redshift gives resolved do...,3,disagreement in z and flag. 2/3 of inspectors ...
145,35185760076305772,GALAXY,0.009,TWL,3.5,R,0.7027,--,--,0.7027,GALAXY,3.0,2,0.009,0.687611,--|dubious fit + VI redshift gives resolved do...,3,disagreement in z and flag. 2/3 of inspectors ...


In [227]:
print(vi.loc[ 45]['merger comment'])

disagreement in z and flag. 2/3 of inspectors give VI z based on doublet. secure.


In [228]:
#copy this text to a new cell to resolve the conflict
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 0.7028
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)

,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
45,35185760076305772,-1,0.33.0.dev3890,0,GALAXY,0.009,DRP,3.5,--,--,...,323.774934,0.7028,GALAXY,3.0,2,0.687611,0.009,--|dubious fit + VI redshift gives resolved do...,3,disagreement in z and flag. 2/3 of inspectors ...
95,35185760076305772,-1,0.33.0.dev3890,0,GALAXY,0.009,vrk,3.5,R,0.7028,...,323.774934,0.7028,GALAXY,3.0,2,0.687611,0.009,--|dubious fit + VI redshift gives resolved do...,3,disagreement in z and flag. 2/3 of inspectors ...
145,35185760076305772,-1,0.33.0.dev3890,0,GALAXY,0.009,TWL,3.5,R,0.7027,...,323.774934,0.7028,GALAXY,3.0,2,0.687611,0.009,--|dubious fit + VI redshift gives resolved do...,3,disagreement in z and flag. 2/3 of inspectors ...


conflict ID =  0


In [229]:
#copy this text to a new cell to display the conflict
conflict_id=1
display_conflict(conflict_id, show_spectra=False)


,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
695,35185760105665533,GALAXY,0.0058,TWL,4.0,--,--,--,--,0.0058,GALAXY,3.0,3,0.0058,0.0,--| Strong narrow lines including HeI|Wierd St...,3,flag disagreement. z is secure.
744,35185760105665533,GALAXY,0.0058,ACE,4.0,--,--,--,Strong narrow lines including HeI,0.0058,GALAXY,3.0,3,0.0058,0.0,--| Strong narrow lines including HeI|Wierd St...,3,flag disagreement. z is secure.
794,35185760105665533,GALAXY,0.0058,PNT,4.0,CS,0.0058,STAR,Wierd Star with emission lines detected I do n...,0.0058,STAR,3.0,3,0.0058,0.0,--| Strong narrow lines including HeI|Wierd St...,3,flag disagreement. z is secure.


In [230]:
#copy this text to a new cell to resolve the conflict
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 'GALAXY'

#add a comment
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)

,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
695,35185760105665533,-1,0.33.0.dev3890,0,GALAXY,0.0058,TWL,4.0,--,--,...,22954.251607,0.0058,GALAXY,3.0,3,0.0,0.0058,--| Strong narrow lines including HeI|Wierd St...,3,flag disagreement. z is secure.
744,35185760105665533,-1,0.33.0.dev3890,0,GALAXY,0.0058,ACE,4.0,--,--,...,22954.251607,0.0058,GALAXY,3.0,3,0.0,0.0058,--| Strong narrow lines including HeI|Wierd St...,3,flag disagreement. z is secure.
794,35185760105665533,-1,0.33.0.dev3890,0,GALAXY,0.0058,PNT,4.0,CS,0.0058,...,22954.251607,0.0058,GALAXY,3.0,3,0.0,0.0058,--| Strong narrow lines including HeI|Wierd St...,3,flag disagreement. z is secure.


conflict ID =  1


Let's check the number of conflicts again.

In [231]:
vi_conflict = find_conflicts(vi_gp)

In [232]:
unique_targets = np.unique(vi_conflict['TargetID'].tolist())
print('Targets with problematic VI: ', unique_targets)
print('Total number of conflicts to resolve: ', len(unique_targets))

Targets with problematic VI:  []
Total number of conflicts to resolve:  0


## Woohoo!

## Finally
we need to recompute vi_combined_flag and vi_combined_z, after everything is resolved.

In [233]:
vi['vi_combined_flag'] = vi.groupby('TargetID')['VI class'].transform('mean')
vi['vi_combined_z'] = vi.groupby('TargetID')['best redshift'].transform('mean')


## Now we prepare to write to file. 

### The important columns for the truth table construction are **vi_combined_flag** and **vi_combined_z** 

The truth table should **take the redhift value in vi_combined_z** for all objects with **vi_combined_flag >= 2.5** (for discussion, but this catches the cases where there is some disagreement on the 2/3 boundary).

After the merging process, those two columns will be the same for ALL of the members of each group by TargetID, so it doesn't matter which member we write to file. Taking the 1st one just makes this easy. 


In [234]:
vi.keys()

Index(['TargetID', 'ExpID', 'Spec version', 'Redrock version',
       'Redrock spectype', 'Redrock z', 'VI scanner', 'VI class', 'VI issue',
       'VI z', 'VI spectype', 'VI comment', 'FIBER', 'FLUX_G', 'FLUX_R',
       'FLUX_Z', 'FIBERFLUX_G', 'FIBERFLUX_R', 'FIBERFLUX_Z', 'DELTACHI2',
       'best redshift', 'best spectype', 'vi_combined_flag', 'vi_diff', 'dz',
       'vi_combined_z', 'all VI comments', 'N_VI', 'merger comment'],
      dtype='object')

In [242]:
print(save_filename)

truth_table_ELGs_sets16_20.txt


In [235]:
#vi_gp['Redrock z', 'Redrock spectype', 'best redshift' , 'best spectype', 'vi_combined_flag', 'all VI comments', 'merger comment', 'N_VI', 'DELTACHI2','FIBER','FLUX_G','FLUX_R','FLUX_Z','FIBERFLUX_G','FIBERFLUX_R','FIBERFLUX_Z'].first().to_csv(VI_dir+'merged_VI.txt')
vi_gp['Redrock z', 'Redrock spectype', 'best redshift' , 'best spectype', 'vi_combined_flag', 'N_VI', 'all VI comments', 'merger comment','DELTACHI2','FIBER','FLUX_G','FLUX_R','FLUX_Z','FIBERFLUX_G','FIBERFLUX_R','FIBERFLUX_Z'].first().to_csv(VI_dir+save_filename)

Check that merged file reads in OK - check comments

In [236]:
merged_file = pd.read_csv(VI_dir+save_filename)

In [237]:
merged_file.keys()

Index(['TargetID', 'Redrock z', 'Redrock spectype', 'best redshift',
       'best spectype', 'vi_combined_flag', 'N_VI', 'all VI comments',
       'merger comment', 'DELTACHI2', 'FIBER', 'FLUX_G', 'FLUX_R', 'FLUX_Z',
       'FIBERFLUX_G', 'FIBERFLUX_R', 'FIBERFLUX_Z'],
      dtype='object')

In [238]:
merged_file

,TargetID,Redrock z,Redrock spectype,best redshift,best spectype,vi_combined_flag,N_VI,all VI comments,merger comment,DELTACHI2,FIBER,FLUX_G,FLUX_R,FLUX_Z,FIBERFLUX_G,FIBERFLUX_R,FIBERFLUX_Z
0,35185760072107938,1.6966,GALAXY,1.6966,GALAXY,0.333333,3,--|continuum only + no signal| Low S/N,none,2.103210,911,0.755319,0.896726,1.247660,0.319496,0.379310,0.527753
1,35185760072108290,1.5248,GALAXY,1.5248,GALAXY,0.666667,3,--| dubious fit (very weak doublet in noisy re...,none,13.945422,902,0.352810,0.282233,0.738608,0.147532,0.118020,0.308859
2,35185760072108293,1.1186,GALAXY,1.1186,GALAXY,4.000000,3,--|--|--,none,363.574735,928,0.616775,1.103485,3.180738,0.199836,0.357532,1.030566
3,35185760072108452,0.7814,GALAXY,0.7814,GALAXY,0.000000,3,--| calibration problem at 6400AA| Calibration...,none,28.326496,921,0.311091,0.361539,0.509834,0.183538,0.213301,0.300792
4,35185760072108734,0.5750,GALAXY,0.5750,GALAXY,0.000000,3,--| dubious fit (doublet and OIII lines in abs...,none,0.192582,610,0.254805,0.229173,0.115708,0.198195,0.178258,0.090001
5,35185760072108765,1.5390,GALAXY,1.5390,GALAXY,1.666667,3,--| small broad doublet in noisy region + no s...,none,31.487196,934,0.218690,0.248305,0.312420,0.170358,0.193427,0.243373
6,35185760072108841,0.8500,GALAXY,0.8500,GALAXY,4.000000,3,--|--| triple peak [OII] emission,none,225.733352,906,0.745091,1.562807,4.487094,0.236130,0.495277,1.422026
7,35185760072108921,1.6269,GALAXY,1.6269,GALAXY,1.666667,3,--| strong doublet but in noise right at the e...,none,12.682689,909,0.225657,0.185179,0.151407,0.175453,0.143981,0.117722
8,35185760072108985,0.7530,GALAXY,0.7530,GALAXY,4.000000,3,--|--| Two galaxies; another one at z=0.8163; ...,none,46.152147,907,0.455895,0.884531,2.136291,0.244703,0.474774,1.146659
9,35185760072109572,0.0091,GALAXY,0.0091,GALAXY,0.666667,3,--| dubious fit (no doublet strong OIII lines ...,none,580.915410,946,0.326185,0.328440,0.654891,0.200175,0.201559,0.401898


In [239]:
merged_file.loc[9]['merger comment']

'none'

In [240]:
merged_file.loc[9]['all VI comments']

'--| dubious fit (no doublet strong OIII lines in absorption) + nothing better from VI + no signal| Model fits sky residuals'

In [241]:
VI_dir

'/global/homes/r/rtojeiro/SV/VI_files/SV0/ELG/'